## SquamataAssemballyAMT - Jupyter notebook for batch releasing Audio Magnetotellic (AMT) data to ScienceBase

This module performs the following operations:
- Create list of data directories.
- Identify files accompanying data release.
- Create file listing for metadata XML markup.
- Identify and load MT EDI file.
- Clean up and reformat harvested values to be XML metadata complient.
- Create User Editable Keywords Listing
- Create entity and attribute XML markup.
- Poplulate metadata template
- Validate metadata; create error log; create HTML and FGDC Text versions of the metadata. (In development - use https://mrdata.usgs.gov/validation/ for validation in the interim)

Known issues needing repair:
- Fix procstep section; do we need this function to collect file information or do we plan on handeling this with boilerplate.

## Future development plans for SquamataSB

- Create all child metadata files from first example created in previous steps. 
- Batch upload files to ScienceBase.
- Batch remove files from ScienceBase. 
- Change ScienceBase parameters such as citation information, add orcid ids, add USGS CMS tags, etc. 

### Instructions
- Create a template xml format that contains boilerplate text common to all childeren in a data release.  Be sure this template contains the approriate curly bracket tags, {SquamataTagExample} used to populate the template using SquamataAMT.

### To execute a function/command select a cell and Hold-Shift + Press-Enter

**The 'r' signifies a string literal. Use for paths.**

Metadata wizard:  Advanced, Open In a jupyter Notebook?
Metadata Wizard 2.o from ScienceBase

v1.1 
- created stack loop for batch mode processing
- harvest process dates
- seperatate out different text fils for A+E, cross power, fourier, etc.
- correct boiler plate for A+E and process steps

v1.2
- fix procedure date harvest bug, typo in first print statement

v1.3
- fix procedure date harvest to harvest file modified date instaed of file created date using "m" time instaed of "c" time; e.g. we want, time.ctime(os.path.getmtime(avgList[0])) NOT time.ctime(os.path.getctime(avgList[0]))
- Fix attribute description boiler plate for BP and SD text files.
- Add entity and attributes for edi file
- add entity and attributes for BP and SD files
- correct loops to create entity and attributes for all FC and TC files, not just the last one in the loop.

v1.4
 - fix attribute description boiler plate for BP.txt and SD.txt (removed the word 'binary')
 - fix attribute description boiler plate for RSP files (BF's units are Volts/Gamma, EF's units are Volts)
 - fix attribute description boiler plate for FC files (replaced word 'Time Series' with 'Fourier Coefficient')
 - fix attribute description boiler plate for BP files (replaced word 'Time Series' with 'Cross Power')
 
v1.5
 - assign the edi file name to variable 'EdiFileName' so it is correctly harvested by the XML template {EdiFileName} tag. 
 - Add code to harvest TS1 file modified time,  {CollectionDate} for "Acquire multiple high frequency time series files TS1 file extension)..." process step. 


In [172]:
# Phil Brown (pbrown@usgs.gov) 2019 Beta
# Working Python 3 Notebook used to facilitate the release of Audio Magnetotelluric (AMT) Data to ScienceBase.

In [173]:
# Test Cell
print ("Jupyter is working.") #To run this cell, hold down Shift and press Enter.

Jupyter is working.


In [1]:
# Load required Libraries
import sys
import os
import zipfile
import csv
#import pysb
import requests
import shutil
from shutil import copyfile
import zipfile
import datetime
import glob
from lxml import etree
import json
import pickle
import shutil
import fileinput
import json
import pandas as pd
import numpy as np
from IPython.core.display import display
from IPython.core.display import HTML
from lxml import etree
##from pymdwizard.core.xml_utils import XMLRecord
##from pymdwizard.core.xml_utils import XMLNode
import re
from ipywidgets import *
from IPython.display import display
from IPython.html.widgets import widgets
import datetime
import dateutil.parser
import time
from IPython.display import Javascript

C:\Users\pbrown\AppData\Local\Continuum\anacondaNew\lib\site-packages\IPython\html.py:14: ShimWarning: The `IPython.html` package has been deprecated since IPython 4.0. You should import from `notebook` instead. `IPython.html.widgets` has moved to `ipywidgets`.
  "`IPython.html.widgets` has moved to `ipywidgets`.", ShimWarning)


# 1) Step One - Set Directory Paths
## Please set directory paths below
### Directory paths include
- Data Path: This is the path to the data, data structure should have a directory for each station
- Template Path: The path to the XML metadata template being used for the data.  This template should already include all information common to all child metadata files e.g. originators, larger work citation, etc.

In [2]:
#Set Data Paths - perhaps we'll get a user form to do this some day?
mtDataPath = r"C:\CurrentWork\DataReleases\AMT_Silverton" #The 'r' signifies a string literal. Use for paths.
mtMetaDataTemplatePath = r"C:\CurrentWork\DataReleases\AMT_Silverton"
mtMetaDataTemplateName = "MT-MetaData_SilvertonTemplate-Brian.xml"

In [3]:
#Check Paths for the fun of it
print ('The MT Data Path is: ' + '"' + mtDataPath + '"')
mtMetaDataTemplatePath + "\\" + mtMetaDataTemplateName

The MT Data Path is: "C:\CurrentWork\DataReleases\AMT_Silverton"


'C:\\CurrentWork\\DataReleases\\AMT_Silverton\\MT-MetaData_SilvertonTemplate-Brian.xml'

## Now, let's explore our data. 
- What files do we have? 
- What files do we import values from?

In [4]:
#Explore data files and directory structure hosted below the provided provided parent data directory

In [5]:
#Produce directory listing of station (SB Object Children)
#Either set up the root directory with station subdirectories only or delete non-station directories from the list array
#mtDataDirList = os.walk(mtDataPath)
#mtDataDirList = [entry.path for entry in os.scandir(mtDataPath) if entry.is_dir()]
mtDataDirList = next(os.walk(mtDataPath))[1]
mtDataDirList

['CC01']

# We can select an indivdual station to test or run code for all stations in batch mode.  

In [6]:
#!!!!!!!!!Run this code and select A single station !!!!!!!!!
#!!!!!!!!!Skip the loop cell below and run cells sequentually !!!!!!!!!!
#!!!!!!!!!Use Shift-Enter to then execute each cell one by one!!!!!!!!!! 
mtStationPath = mtDataPath + '\\' + mtDataDirList[0]
mtStationPath

'C:\\CurrentWork\\DataReleases\\AMT_Silverton\\CC01'

# -----------------------------------------------------------------------------------------------------------

In [9]:
#!!!!!!!!!!!!!!!!!!  Run this code to loop through ALL Data Sets !!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!
#!!!!!!!This code will produce metadata files for ALL stations in batch mode!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!
#!!!!!!!!!!!!!!!!!!  Skip running this code if processing a single station !!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!
for i in range (0,len(mtDataDirList)):
    mtStationPath = mtDataPath + '\\' + mtDataDirList[i]
    from IPython.display import Javascript
    display(Javascript('IPython.notebook.execute_cells_below()'))

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

...

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

# -----------------------------------------------------------------------------------------------------------

In [7]:

#Look for EDI file to load
ediList = glob.glob(os.path.join(mtStationPath, '**/*MT*.edi'),  recursive=True)
ediPath = ediList[0]
#ediList
print ('EDI File Path:\n' + ediPath)
ediPathArray = ediPath.split('\\')
ediFile = str(ediPathArray[len(ediPathArray)-1])
print ('EDI File:\n' + ediFile)
EdiFileName = ediFile
#ediPathArray

EDI File Path:
C:\CurrentWork\DataReleases\AMT_Silverton\CC01\USA-Colorado-Silverton_Caldera-2018-AMTCC01.edi
EDI File:
USA-Colorado-Silverton_Caldera-2018-AMTCC01.edi


## Harvest from the MT EDI file. 
### Parameters include:
- ProductId=USA-New_Mexico-Rio_Grande_Rift-San_Luis_Basin-2009-AMT01.edi
- ExternalUrl Url=https://doi.org/10.5066/F72F7MQ7
- Attachment Filename=https://pubs.usgs.gov/of/2011/1264/report/OF11-1264.pdf
- Survey Purpose Description: 
- Data Description:
- Citation Title=Audiomagnetotelluric data, Taos Plateau Volcanic Field, New Mexico
- Citation Authors=Chad E. Ailes, Brian D. Rodriguez
- Citation Year=2011
- YearCollected=2009
- Country=USA                                  
- Ellipsoid=Clarke 1866                                                          
- Location datum=NAD27 CONUS                                                     
- SITE LATITUDE=36.752985000                                                     
- SITE LONGITUDE=-105.560966167                                                  
- Elevation units="meters"=2608.00                                                                     
- Start=2009-07-21T19:52:03 UTC/GMT
- End=2009-07-21T20:34:20 UTC/GMT
- ProcessingTimeSeriesUsed:
         wp01A1.bp1                                                                     
         wp01A2.bp1                                                                     
         wp01A1.sd6                                                                     
         wp01A2.sd6                                                                     
         wp01A1.sd7                                                                     
         wp01A2.sd8                                                                     
         wp01A2_3.sd9 
- Entities and Attributes:
    - FREQUENCIES
    - IMPEDANCE ROTATION ANGLES
    - IMPEDANCES
    - TIPPER PARAMETERS
    - COMPUTED PARAMETERS


## Lets now import and index values from the EDI Files
- We need these values for the metadata template.  
- We also want to run stats on some of these values for the entity and attributes section

In [8]:
    #Load EDI File and Read It
    ediFile = open(ediPath, 'r')
    ediContent = ediFile.read()
    ediFile.close()
    print(ediContent)


>HEAD                                                                           
                                                                                
  DATAID="Cunningham Creek"                                                     
  ACQBY=USGS                                                                    
  ACQDATE=2018-07-17
  STATE=Colorado                                                                
  COUNTY="San Juan"                                                             
  UNITS=M                                                                       
  STDVERS=1.0                                                                   
  PROGVERS=GEOTOOLS_2.3                                                         
  PROGDATE=09/16/94                                                             
                                                                                
>INFO   MAXLINES=1000                                                           
       

In [9]:
#Now assign values to the SB MetaDataWizard Template unknowns
list_ = ediContent.splitlines()
list_length = len (list_)

# there are probally easier ways to loop through the below but I like having it all hard coded upfront
# it's easire to track an change for me
# Use the examples provided below to extract additional parameters
# Not that all variables being collected are not necessarily used in populating the template.
# Note that values can be hardcoded into the metadata xml template and/or harvested from the edi file

for X in list_:
  if "ProductId" in X:
    productArray = X.split('=')
    productIdArray = productArray[1].split('.')
    productId = productIdArray[0]
    # We may want to reformat this are parse out this name further for use with a root name based on the Data Release Title?
    productId = productId.replace("-", " ")
    productId = productId.replace("_", " ")
    drTitle = productId
    print ('Child Title: ' + productId)
  if "ExternalUrl Url" in X:
    externalURLArray = X.split('=')
    externalURL = externalURLArray[1]
    print ('<onlink>: ' + externalURL)
  if "STATE" in X:
    stateArray = X.split('=')
    state = stateArray[1].replace('"', "") #remove quotes around state
    print ('State: ' + state)
  if "COUNTY" in X:
    countyArray = X.split('=')
    county = countyArray[1]
    print ('County: ' + county)
  if "Start" in X:
    startArray = X.split('=')
    start = startArray[1]
    print ('Start: ' + start)
  if "End" in X:
    endArray = X.split('=')
    end = endArray[1]
    print ('End: ' + end)
  if "Attachment Filename" in X and "http" in X:
    lgwrklinkArray = X.split('=')
    lgwrklink = lgwrklinkArray[1]
    print ('Attachment Filename Link: ' + lgwrklink)
  if "Citation Title" in X:
    citTitArray = X.split('=')
    citTit = citTitArray[1]
    print ('Citation Title: ' + citTit)
  if "Citation Authors" in X:
    citNamesArray = X.split('=')
    citAuthorsArray = citNamesArray[1].split(',')
    for author in citAuthorsArray:
     author = author.strip()
     print ('Author: '+ author)
  if "Citation Year" in X:
    citYearArray = X.split('=')
    citYear = citYearArray[1]
    print ('Citation Year: ' + citYear)
  if "YearCollected" in X:
    yearColArray = X.split('=')
    yearCol = yearColArray[1]
    print ('Year Collected: ' + yearCol)
  if "Ellipsoid" in X:
    ellipsoidArray = X.split('=')
    ellipsoid = ellipsoidArray[1]
    print ('Ellipsoid: ' + ellipsoid)
  if "Location datum" in X:
    locDatumArray = X.split('=')
    locDatum = locDatumArray[1]
    print ('Local datum: ' + locDatum)
  if "SITE LATITUDE" in X:
    sitLatArray = X.split('=')
    sitLat = sitLatArray[1] # !!! probally need to reformat this to have only 6 significant digits Also need to trim extra spaces !!!
    print ('Site latitude: ' + sitLat)
  if "SITE LONGITUDE" in X:
    sitLonArray = X.split('=')
    sitLon = sitLonArray[1] # !!! probally need to reformat this to have only 6 significant digits Also need to trim extra spaces !!!
    print ('Site longitude: ' + sitLon)
  if "Elevation units" in X:
    elevationStringArray = X.split('=')
    siteElevation = elevationStringArray[2] 
    print ('Site Elevation: ' + siteElevation)
    elevationUnits = elevationStringArray[1].replace('"', "")
    print ('Elevation Units: ' + elevationUnits)
    
# Code below returns values that occupy more than one line
    
for i in range(list_length):
 value = list_[i] 
 if value.replace(" ", "") == 'SurveyPurposeDescription:':
   startIndPurpose = i + 1
   #print ('startIndPurpose: ' + str(startIndPurpose))
 if value.replace(" ", "") == 'DataDescription:':
   endIndPurpose = i - 1
   #print ('endIndPurpose: ' + str(endIndPurpose))
purpose = list_[startIndPurpose]
for j in range(startIndPurpose + 1,endIndPurpose): 
    purpose = purpose + list_[j]
    purposeClean = re.sub(' +', ' ',purpose)
print ('\nAbstract:\n\t' + purposeClean)

for k in range(list_length):
 value = list_[k] 
 if value.replace(" ", "") == 'DataDescription:':
   startIndDescription = k + 1
   #print ('startIndDescription: ' + str(startIndDescription))
 if value.replace(" ", "") == 'FILECREATOR:':
   endIndDescription = k - 9
   #print ('endIndDescription: ' + str(endIndDescription))
description = list_[startIndDescription]
for l in range(startIndDescription + 1,endIndDescription): 
    description = description + list_[l]
    descriptionClean = re.sub(' +', ' ',description)
print ('\nPurpose:\n\t' + descriptionClean)
    

State: Colorado                                                                
County: "San Juan"                                                             
Child Title: USA Colorado Silverton Caldera 2018 AMTCC01
<onlink>: https://doi.org/10.5066/P9PPQTJ8 
Citation Title: Audiomagnetotelluric sounding data in the Silverton Caldera complex, Colorado, 2018
Author: James Reitman
Author: Brian D. Rodriguez
Author: and Philip J. Brown
Author: II.
Citation Year: 2019                                                             
Year Collected: 2018
Ellipsoid: Clarke 1866                                                          
Local datum: NAD27 CONUS                                                     
Site latitude: 37.83424                                                         
Site longitude: -107.59897                                                      
Site Elevation: 2962.00                                               
Elevation Units: meters
Start: 2018-07-17T19:48:36 UTC/G

In [10]:
#Reformat product ID to more correctly reflect child (station) name
arrChildID = productId.split(' ')
strChildID = arrChildID[-1]
strChildID


'AMTCC01'

In [11]:
# Now let's format the start time and end time to be what the XML file wants for <begdate> and <enddate>

begdateArr = start.split(' ')
begdate_str = begdateArr[0]
begdate_obj = dateutil.parser.parse(begdate_str)
begdate = begdate_obj.strftime('%Y%m%d')
print('<begdate> ', begdate) 

enddateArr = end.split(' ')
enddate_str = enddateArr[0]
enddate_obj = dateutil.parser.parse(enddate_str)
enddate = enddate_obj.strftime('%Y%m%d')
print('<enddate> ', enddate) 


<begdate>  20180717
<enddate>  20180717


In [12]:
#Now we reformat the lat  and longitude to 6 sig figs as well as trim of any extra spaces
#Brian seems to be stripping out the sig figs now so there is no need.  
#Also may whant to round up instead of just stripping values?

sitLat = sitLat.strip()
##sitLat = sitLat[:-3] 
sitLon = sitLon.strip()
##sitLon = sitLon[:-3]

print ('Site latitude: ' + sitLat)
print ('Site longitude: ' + sitLon)

Site latitude: 37.83424
Site longitude: -107.59897


In [13]:
# Now Reformat county by trimming the extra spaces
county = county.strip()
print ('County: ' + county)

County: "San Juan"


In [14]:
## Create editable keywords example.  
## Example text is created after running this cell
## This text is displayed by running "display(keywords) below
keywords = widgets.Textarea(
    value='\t\t<keywords>\n\t\t\t<theme>\n\t\t\t\t<themekt>ISO 19115 Topic Category</themekt>' \
    + '\n\t\t\t\t<themekey>biota</themekey>\n\t\t\t</theme>\n\t\t\t<theme>\n\t\t\t\t<themekt>None</themekt>' \
    + '\n\t\t\t\t<themekey>impedance</themekey>\n\t\t\t\t<themekey>tipper</themekey>' \
    + '\n\t\t\t\t<themekey>apparent resistivity</themekey>\n\t\t\t\t<themekey>impedance phase</themekey>' \
    + '\n\t\t\t\t<themekey>impedance strike</themekey>\n\t\t\t\t<themekey>MT</themekey>' \
    + '\n\t\t\t\t<themekey>audiomagnetotelluric</themekey>\n\t\t\t\t<themekey>magnetotelluric</themekey>' \
    + '\n\t\t\t\t<themekey>AMT</themekey>\n\t\t\t\t<themekey>sounding</themekey>' \
    + '\n\t\t\t\t<themekey>Geology, Geophysics, and Geochemistry Science Center</themekey>' \
    + '\n\t\t\t\t<themekey>GGGSC</themekey>\n\t\t\t\t<themekey>Mineral Resources Program</themekey>' \
    + '\n\t\t\t\t<themekey>MRP</themekey>\n\t\t\t</theme>\n\t\t\t<theme>\n\t\t\t\t<themekt>USGS Thesaurus</themekt>' \
    + '\n\t\t\t\t<themekey>Magnetic field (earth)</themekey>\n\t\t\t\t<themekey>Geophysics</themekey>' \
    + '\n\t\t\t\t<themekey>GPS measurement</themekey>\n\t\t\t\t<themekey>Electromagnetic surveying</themekey>' \
    + '\n\t\t\t\t<themekey>Magnetic surveying</themekey>\n\t\t\t</theme>\n\t\t\t<place>' \
    + '\n\t\t\t\t<placekt>USGS Geographic Names Information System (GNIS), https://geonames.usgs.gov</placekt>' \
    + '\n\t\t\t\t<placekey>Colorado</placekey>\n\t\t\t\t<placekey>Silverton</placekey>' \
    + '\n\t\t\t\t<placekey>' + county + ' County</placekey>\n\t\t\t</place>\n\t\t</keywords>',
    placeholder='Type something',
    #description='String:',
    layout=Layout(width='100%', height='666px'),
    disabled=False
)
print ('Keywords list created.')

Keywords list created.


### Change the text in the textbox below to relflect what should be included as the key words for all child items

Note that changing the text below at any time creates a keywords section of the metadata seen EXACTLY as it is shown below

In [15]:
# Run this cell for key word text to edit.  
# Edit the text in place.  
# When complete move on to the next step

display(keywords)

Textarea(value='\t\t<keywords>\n\t\t\t<theme>\n\t\t\t\t<themekt>ISO 19115 Topic Category</themekt>\n\t\t\t\t<themekey>biota</themekey>\n\t\t\t</theme>\n\t\t\t<theme>\n\t\t\t\t<themekt>None</themekt>\n\t\t\t\t<themekey>impedance</themekey>\n\t\t\t\t<themekey>tipper</themekey>\n\t\t\t\t<themekey>apparent resistivity</themekey>\n\t\t\t\t<themekey>impedance phase</themekey>\n\t\t\t\t<themekey>impedance strike</themekey>\n\t\t\t\t<themekey>MT</themekey>\n\t\t\t\t<themekey>audiomagnetotelluric</themekey>\n\t\t\t\t<themekey>magnetotelluric</themekey>\n\t\t\t\t<themekey>AMT</themekey>\n\t\t\t\t<themekey>sounding</themekey>\n\t\t\t\t<themekey>Geology, Geophysics, and Geochemistry Science Center</themekey>\n\t\t\t\t<themekey>GGGSC</themekey>\n\t\t\t\t<themekey>Mineral Resources Program</themekey>\n\t\t\t\t<themekey>MRP</themekey>\n\t\t\t</theme>\n\t\t\t<theme>\n\t\t\t\t<themekt>USGS Thesaurus</themekt>\n\t\t\t\t<themekey>Magnetic field (earth)</themekey>\n\t\t\t\t<themekey>Geophysics</themekey>\n\t\t\t\t<themekey>GPS measurement</themekey>\n\t\t\t\t<themekey>Electromagnetic surveying</themekey>\n\t\t\t\t<themekey>Magnetic surveying</themekey>\n\t\t\t</theme>\n\t\t\t<place>\n\t\t\t\t<placekt>USGS Geographic Names Information System (GNIS), https://geonames.usgs.gov</placekt>\n\t\t\t\t<placekey>Colorado</placekey>\n\t\t\t\t<placekey>Silverton</placekey>\n\t\t\t\t<placekey>"San Juan" County</placekey>\n\t\t\t</place>\n\t\t</keywords>', layout=Layout(height='666px', width='100%'), placeholder='Type something')

### The module below collects the file date property for use in creating the process description dates

In [16]:
# module to get file process dates for process descriptions
# Visit https://docs.python.org/2/library/datetime.html#strftime-and-strptime-behavior for time format standards
# Note that get c time gives creation date.  Get m time gives creation date.
# We want cration time e.g. we want, time.ctime(os.path.getmtime(avgList[0])) NOT time.ctime(os.path.getctime(avgList[0]))

# Set {CollectionDate} to the station end date
strCollectionDate = enddate
print ("{CollectionDate} " + strCollectionDate)

# {CombinationDate} Get the Time stamp for the AVG file
# First we need to get the file path
avgList = glob.glob(os.path.join(mtStationPath, '*.AVG'),  recursive=True)
#print (avgList)
avgTime = time.ctime(os.path.getmtime(avgList[0]))
print ("avgTime: " + avgTime)
avgTime_obj = datetime.datetime.strptime(avgTime,'%a %b %d %H:%M:%S %Y')
strCombinationDate = datetime.datetime.strftime(avgTime_obj,'%Y%m%d')
print ("{CombinationDate} " + strCombinationDate)


# {ConversionDate} is the date of the older edi file, this is the second file in the edi list
# No that isn't it, I filtered to get the newer edi file, oh I'll just find the file 
# in an array of edi files that isn't the other one

ediAllList = glob.glob(os.path.join(mtStationPath, '*.edi'),  recursive=True)
#print (ediAllList)
for i in range (0,len(ediAllList)):
  if ediAllList[i] != ediPath:
    OriginalEDIPath = ediAllList[i]
os.path.getctime(OriginalEDIPath)
oediTime = time.ctime(os.path.getmtime(OriginalEDIPath))
print ("oediTime: " + oediTime)
oediTime_obj = datetime.datetime.strptime(oediTime,'%a %b %d %H:%M:%S %Y')
strConversionDate = datetime.datetime.strftime(oediTime_obj,'%Y%m%d')
print ("{ConversionDate} " + strConversionDate)

# {RotationDate} Get the Time stamp for the AVG file
# First we need to get the file path
dmpList = glob.glob(os.path.join(mtStationPath, '*.dmp'),  recursive=True)
#print (dmpList)
dmpTime = time.ctime(os.path.getmtime(dmpList[0]))
print ("dmpTime: " + dmpTime)
dmpTime_obj = datetime.datetime.strptime(dmpTime,'%a %b %d %H:%M:%S %Y')
strRotationDate = datetime.datetime.strftime(dmpTime_obj,'%Y%m%d')
print ("{RotationDate} " + strRotationDate)

#{HarvestDate} = final edi file 
os.path.getctime(ediPath)
filetime = time.ctime(os.path.getmtime(ediPath))
print ("Harvest Time: " + filetime)
filetime_obj = datetime.datetime.strptime(filetime,'%a %b %d %H:%M:%S %Y')
strHarvestDate = datetime.datetime.strftime(filetime_obj,'%Y%m%d')
print ("{HarvestDate} " + strHarvestDate)

{CollectionDate} 20180717
avgTime: Tue Dec 18 16:09:58 2018
{CombinationDate} 20181218
oediTime: Wed Feb  6 14:51:34 2019
{ConversionDate} 20190206
dmpTime: Tue Feb  5 15:24:48 2019
{RotationDate} 20190205
Harvest Time: Wed Feb 13 16:31:06 2019
{HarvestDate} 20190213


Entity and Attribute Values for the EDI file.  List !****FREQUENCIES****!,!****IMPEDANCE ROTATION ANGLES****!,!****IMPEDANCES****!,!****COMPUTED PARAMETERS****!

Here we load the frequencies
>!****FREQUENCIES****!

In [17]:
# Import entity and attributes - !****FREQUENCIES****! plan to break some of these individual chunks into objects/functions

# Get Range of Frequency Values in EDI File
for k in range(list_length):

 value = list_[k] 
 if value.replace(" ", "") == '>!****FREQUENCIES****!':
   startIndFrequencies = k + 2
   print ('startIndFrequencies: ' + str(startIndFrequencies))
 
 if value.replace(" ", "") ==  '>!****IMPEDANCEROTATIONANGLES****!':
   endIndFrequencies = k - 1
   print ('endIndFrequencies: ' + str(endIndFrequencies))

frequencyData = []
fdata = []
fdataTemp = []
#Constuct a library of Headers and Values using Pandas, https://pandas.pydata.org/
frequencyDF = pd.DataFrame(fdata)
for j in range(startIndFrequencies,endIndFrequencies):
    fdataTemp = list_[j]
    fdataTemp = re.sub(' +', ' ',fdataTemp)
    fdataTemp = fdataTemp.split(" ")
    del fdataTemp[0]
    fdata = fdata + fdataTemp
    
print (fdata)  
fdata = np.array(fdata).astype(np.float) #convert String to floats
frequencyDF = pd.DataFrame(fdata,columns=['Frequencies'])
frequencyDF


startIndFrequencies: 272
endIndFrequencies: 279
['2.33700000E+04', '1.52900000E+04', '1.15900000E+04', '6.85000000E+03', '5.21000000E+03', '3.55000000E+03', '2.73000000E+03', '2.42000000E+03', '1.87000000E+03', '1.17000000E+03', '8.85000000E+02', '7.20000000E+02', '5.80000000E+02', '4.60000000E+02', '3.40000000E+02', '2.70000000E+02', '2.10000000E+02', '1.72399994E+02', '1.60000000E+02', '1.40000000E+02', '1.22099998E+02', '1.00000000E+02', '8.59400024E+01', '7.90000000E+01', '6.00600014E+01', '4.15000000E+01', '2.83199997E+01', '1.90400009E+01', '1.22100000E+01', '7.32399988E+00', '4.39400005E+00']


,Frequencies
0,23370.000000
1,15290.000000
2,11590.000000
3,6850.000000
4,5210.000000
5,3550.000000
6,2730.000000
7,2420.000000
8,1870.000000
9,1170.000000


In [18]:
# Now lets get the stats of the frequency data
#Make Array of Max Vallues
frequencyMax = frequencyDF[('Frequencies')].max()
print ('Max. Frequency: ' + str(frequencyMax))
frequencyMin = frequencyDF[('Frequencies')].min()
print ('Min. Frequency: ' + str(frequencyMin))

Max. Frequency: 23370.0
Min. Frequency: 4.39400005


Here we load the Impedance Rotation Angles
>!****IMPEDANCE ROTATION ANGLES****!

In [19]:
# Import entity and attributes - !****IMPEDANCE ROTATION ANGLES****! plan to break some of these individual chunks into objects/functions
# Get Range of IMPEDANCE ROTATION ANGLES in EDI File
for k in range(list_length):
 value = list_[k] 
 if value.replace(" ", "") == '>!****IMPEDANCEROTATIONANGLES****!':
   startIndROT = k + 2
   print ('startIndROT: ' + str(startIndROT))
 
 if value.replace(" ", "") ==  '>!****IMPEDANCES****!':
   endIndROT = k - 1
   print ('endIndROT: ' + str(endIndROT))

rdata = []
rdataTemp = []
#Constuct a library of Headers and Values using Pandas, https://pandas.pydata.org/
rotationDF = pd.DataFrame(rdata)
for j in range(startIndROT,endIndROT):
    rdataTemp = list_[j]
    rdataTemp = re.sub(' +', ' ',rdataTemp)
    rdataTemp = rdataTemp.split(" ")
    del rdataTemp[0]
    rdata = rdata + rdataTemp
    
print (rdata)  
rdata = np.array(rdata).astype(np.float) #convert String to floats
rotationDF = pd.DataFrame(rdata,columns=['ZROT'])
rotationDF

startIndROT: 282
endIndROT: 289
['0.00000000E+00', '0.00000000E+00', '0.00000000E+00', '0.00000000E+00', '0.00000000E+00', '0.00000000E+00', '0.00000000E+00', '0.00000000E+00', '0.00000000E+00', '0.00000000E+00', '0.00000000E+00', '0.00000000E+00', '0.00000000E+00', '0.00000000E+00', '0.00000000E+00', '0.00000000E+00', '0.00000000E+00', '0.00000000E+00', '0.00000000E+00', '0.00000000E+00', '0.00000000E+00', '0.00000000E+00', '0.00000000E+00', '0.00000000E+00', '0.00000000E+00', '0.00000000E+00', '0.00000000E+00', '0.00000000E+00', '0.00000000E+00', '0.00000000E+00', '0.00000000E+00']


,ZROT
0,0.0
1,0.0
2,0.0
3,0.0
4,0.0
5,0.0
6,0.0
7,0.0
8,0.0
9,0.0


In [20]:
# Now lets get the stats of the rotation data

#Make Array of Max Values
rotationMax = rotationDF[('ZROT')].max()
print ('Max. ZROT: ' + str(rotationMax))

#Make Array of Min Values
rotationMin = rotationDF[('ZROT')].min()
print ('Min. ZROT: ' + str(rotationMin))

Max. ZROT: 0.0
Min. ZROT: 0.0


Here we load the impedances
>!****IMPEDANCES****!

In [21]:
# Import entity and attributes - !****IMPEDANCES****! plan to break some of these individual chunks into objects/functions
# Get Range of Impedance Values in EDI File
for k in range(list_length):
 value = list_[k] 
 if value.replace(" ", "") == '>!****IMPEDANCES****!':
   startIndImpedances = k + 1
   print ('startIndImpedances: ' + str(startIndImpedances))
 
 if value.replace(" ", "") ==  '>!****TIPPERPARAMETERS****!':
   endIndImpedances = k - 1
   print ('endIndImpedances: ' + str(endIndImpedances))

#Construct Array of Channel Headers   
count = 0
impedanceLabel = []
impedanceData = []
data = []
#Constuct a library of Headers and Values using Pandas, https://pandas.pydata.org/
impedanceDF = pd.DataFrame(data)
for l in range(startIndImpedances,endIndImpedances): 
    if list_[l][0] == '>':
     temp = list_[l].split(" ", 1)
     #print (temp)
     impedanceLabel.append((temp[0].split(">"))[1])
     dataTemp = list_[l+1]
     for j in range(l+2,l+8):
      dataTemp = dataTemp + list_[j]
      dataTemp = re.sub(' +', ' ',dataTemp)
     data = dataTemp.split(" ")
     del data[0] # need to check for empty strings and delete these from the array of the string can't be converted to a float
     del data[len(data)-1] # need to check for empty strings and delete
     #print (data)
     data = np.array(data).astype(np.float) #convert String to floats
     se = pd.Series(data)
     print ((temp[0].split(">"))[1])   
     impedanceDF[((temp[0].split(">"))[1])] = se.values
    
    count = count + 1

#impedanceDF = pd.DataFrame(data, columns=(impedanceLabel))
impedanceDF
#data
#se 

startIndImpedances: 291
endIndImpedances: 398
ZXXR
ZXXI
ZXX.VAR
ZXYR
ZXYI
ZXY.VAR
ZYXR
ZYXI
ZYX.VAR
ZYYR
ZYYI
ZYY.VAR


,ZXXR,ZXXI,ZXX.VAR,ZXYR,ZXYI,ZXY.VAR,ZYXR,ZYXI,ZYX.VAR,ZYYR,ZYYI,ZYY.VAR
0,3281.913330,2098.523440,438342.500000,-12294.281300,-6821.877440,589976.125000,6673.753420,5087.249510,308491.844000,-7391.857420,-3054.106930,415206.875000
1,1520.203130,99.280197,47314.113300,-4134.078610,903.713013,76537.445300,-743.059692,-2109.185550,52176.875000,-4389.758300,-279.235748,84403.664100
2,-138.586273,-2253.285640,26264.439500,-2837.942630,1450.599490,60239.648400,2340.813480,-395.997864,36432.750000,677.798828,855.902893,83561.500000
3,1656.330930,-1017.653990,13680.297900,3823.169920,233.396973,25446.896500,-1927.933720,-1226.578490,2626.455320,-1525.655150,813.733032,4885.502930
4,2983.454350,-546.141968,196825.859000,3546.686770,1248.541020,213370.719000,-2809.318120,-1569.066650,55050.418000,-1106.925420,320.498718,59677.867200
5,144.588989,432.338348,179990.828000,1942.602170,3317.719730,138219.094000,-1665.234130,-1521.175900,40261.968800,136.296570,-1163.384770,30918.087900
6,-1069.178830,2160.962650,324984.250000,2012.554440,2893.935060,162708.078000,-847.232178,-1938.151000,181265.078000,501.820313,-1199.597900,90752.984400
7,-208.915405,384.129089,4351.152830,2447.286130,1682.285400,16574.244100,-923.292419,-1068.245970,2563.527340,440.004669,-596.744019,9764.890630
8,-680.091919,504.944611,12263.577100,1923.597530,1606.433840,7351.083500,-1282.313110,-713.982605,10745.893600,428.641083,-452.770782,6441.347660
9,-545.833435,-18.697493,7879.539550,2025.384640,1077.113400,3375.444820,-1146.023930,-540.130249,9005.932620,740.192078,-685.133301,3857.969730


In [22]:
# Now lets get the stats of the impedance data

# Make Array of Max Values
impedanceMax = []
for i in range (0,len(impedanceLabel)):
    impedanceMax.append(impedanceDF[(impedanceLabel[i])].max())
print ('Impedance Max: ' + str(impedanceMax))

# Make Array of Min Values
impedanceMin = []
for i in range (0,len(impedanceLabel)):
    impedanceMin.append(impedanceDF[(impedanceLabel[i])].min())
print ('Impedance Min: ' + str(impedanceMin))

Impedance Max: [3281.91333, 2160.96265, 438342.5, 3823.16992, 3317.71973, 589976.125, 6673.75342, 5087.24951, 308491.844, 2003.59265, 855.902893, 415206.875]
Impedance Min: [-1069.17883, -2253.28564, 552.845947, -12294.2813, -6821.87744, 440.555298, -2809.31812, -2109.18555, 896.376709, -7391.85742, -3054.10693, 341.657623]


In [23]:
#Make Array attributes for impedances
impAtt = ''
for i in range (0,len(impedanceLabel)):
     headerLabel = impedanceLabel[i]
     print ('Header Label: ' + headerLabel + '-' + headerLabel[:3])
     if headerLabel.find('R') != -1:
        impAtti = '\n\t\t<attr>\n\t\t\t\t<attrlabl>' + headerLabel + '</attrlabl>\n' \
        + '\t\t\t\t<attrdef>Real component of impedance ' + headerLabel[:3] + ' in ohm-m (Ex / Hx)</attrdef>' \
        + '\n\t\t\t\t<attrdefs>U.S. Geological Survey</attrdefs>' \
        + '\n\t\t\t\t<attrdomv>\n\t\t\t\t<rdom>\n\t\t\t<rdommin>'+ str(impedanceMin[i]) \
        + '</rdommin>\n\t\t\t\t<rdommax>' + str(impedanceMax[i]) + '</rdommax>\n\t\t\t\t'\
        + '<attrunit>ohm-m</attrunit>\n\t\t\t\t</rdom>\n\t\t\t\t</attrdomv>\n' \
        + '\t\t</attr>'
        impAtt = impAtt + impAtti
     if headerLabel.find('I') != -1:
        impAtti = '\n\t\t<attr>\n\t\t\t<attrlabl>' + headerLabel + '</attrlabl>\n' \
        + '\t\t\t\t<attrdef>Imaginary component of impedance ' + headerLabel[:3] + ' in ohm-m (Ex / Hx)</attrdef>' \
        + '\n\t\t\t\t<attrdefs>U.S. Geological Survey</attrdefs>' \
        + '\n\t\t\t\t<attrdomv>\n\t\t\t\t<rdom>\n\t\t\t<rdommin>'+ str(impedanceMin[i]) + '</rdommin>\n\t\t\t\t\t<rdommax>' \
        + str(impedanceMax[i]) + '</rdommax>\n\t\t\t'\
        + '<attrunit>ohm-m</attrunit>\n\t\t\t\t</rdom>\n\t\t\t\t</attrdomv>\n' \
        + '\t\t</attr>'
        impAtt = impAtt + impAtti
     if headerLabel.find('.VAR') != -1:
        impAtti = '\n\t\t<attr>\n\t\t\t\t<attrlabl>' + headerLabel + '</attrlabl>\n' \
        + '\t\t\t\t<attrdef>Variance of real and imaginary impedance of ' + headerLabel[:3] \
        + ' in ohm-m (Ex / Hx)</attrdef>' \
        + '\n\t\t\t\t<attrdefs>U.S. Geological Survey</attrdefs>' \
        + '\n\t\t\t\t<attrdomv>\n\t\t\t\t\t<rdom>\n\t\t\t<rdommin>'+ str(impedanceMin[i]) + '</rdommin>\n\t\t\t<rdommax>' \
        + str(impedanceMax[i]) + '</rdommax>\n\t\t\t'\
        + '<attrunit>ohm-m</attrunit>\n\t\t\t\t</rdom>\n\t\t\t\t</attrdomv>\n' \
        + '\n\t\t</attr>'
        impAtt = impAtt + impAtti
        
print (impAtt)
ImpedanceAttributes = impAtt

Header Label: ZXXR-ZXX
Header Label: ZXXI-ZXX
Header Label: ZXX.VAR-ZXX
Header Label: ZXYR-ZXY
Header Label: ZXYI-ZXY
Header Label: ZXY.VAR-ZXY
Header Label: ZYXR-ZYX
Header Label: ZYXI-ZYX
Header Label: ZYX.VAR-ZYX
Header Label: ZYYR-ZYY
Header Label: ZYYI-ZYY
Header Label: ZYY.VAR-ZYY

		<attr>
				<attrlabl>ZXXR</attrlabl>
				<attrdef>Real component of impedance ZXX in ohm-m (Ex / Hx)</attrdef>
				<attrdefs>U.S. Geological Survey</attrdefs>
				<attrdomv>
				<rdom>
			<rdommin>-1069.17883</rdommin>
				<rdommax>3281.91333</rdommax>
				<attrunit>ohm-m</attrunit>
				</rdom>
				</attrdomv>
		</attr>
		<attr>
			<attrlabl>ZXXI</attrlabl>
				<attrdef>Imaginary component of impedance ZXX in ohm-m (Ex / Hx)</attrdef>
				<attrdefs>U.S. Geological Survey</attrdefs>
				<attrdomv>
				<rdom>
			<rdommin>-2253.28564</rdommin>
					<rdommax>2160.96265</rdommax>
			<attrunit>ohm-m</attrunit>
				</rdom>
				</attrdomv>
		</attr>
		<attr>
				<attrlabl>ZXX.VAR</attrlabl>
				<attrdef>Real

Here we load the tipper parameters
>!****TIPPER PARAMETERS****!

In [24]:
# Import entity and attributes - !****TIPPER PARAMETERS****! plan to break some of these individual chunks into objects/functions
# Probably will need two functions for this - one for a single list and one for the long lists with more than one column

# Get Range of TIPPER PARAMETERS in EDI File
for k in range(list_length):
 value = list_[k] 
 if value.replace(" ", "") == '>!****TIPPERPARAMETERS****!':
   startIndTipper = k + 1
   print ('startIndTipper: ' + str(startIndTipper))
 
 if value.replace(" ", "") ==  '>!****COMPUTEDPARAMETERS****!':
   endIndTipper = k - 1
   print ('endIndTipper: ' + str(endIndTipper))

#Construct Array of Channel Headers   
count = 0
tipperLabel = []
tipperData = []
tdata = []
#Constuct a library of Headers and Values using Pandas, https://pandas.pydata.org/
tipperDF = pd.DataFrame(tdata)
for l in range(startIndTipper,endIndTipper): 
    if list_[l][0] == '>':
     ttemp = list_[l].split(" ", 1)
     #print (ttemp)
     tipperLabel.append((ttemp[0].split(">"))[1])
     tdataTemp = list_[l+1]
     for j in range(l+2,l+8):
      tdataTemp = tdataTemp + list_[j]
      tdataTemp = re.sub(' +', ' ',tdataTemp)
      tdata = tdataTemp.split(" ")
     #print (tdata)
     del tdata[0]
     del tdata[len(tdata)-1] # need to check for empty strings and delete
     tdata = np.array(tdata).astype(np.float) #convert String to floats
     te = pd.Series(tdata)
     print ((ttemp[0].split(">"))[1])   
     tipperDF[((ttemp[0].split(">"))[1])] = te.values
    
    count = count + 1

#tipperDF = pd.DataFrame(tdata, columns=(tipperLabel))
tipperDF
#tdata
#te 

startIndTipper: 400
endIndTipper: 453
TXR.EXP
TXI.EXP
TXVAR.EXP
TYR.EXP
TYI.EXP
TYVAR.EXP


,TXR.EXP,TXI.EXP,TXVAR.EXP,TYR.EXP,TYI.EXP,TYVAR.EXP
0,-4.452696,-0.580762,0.444237,9.319725,0.057709,0.597910
1,-2.037275,-1.310859,0.370525,9.301649,-0.874892,0.599377
2,-4.194787,-0.077745,0.126227,6.507510,-0.336751,0.289512
3,-1.443829,-0.265709,0.153542,2.916321,-0.299026,0.285606
4,-1.389305,0.484332,1.124407,9.331008,-0.067086,1.218923
5,-3.614978,1.775306,0.991576,3.201192,-0.994398,0.761454
6,-3.097676,-7.326594,89.724609,5.920394,-8.278591,44.921928
7,-2.300843,-0.673272,0.328374,5.796846,-3.923363,1.250830
8,-3.677882,2.254767,0.426927,3.666348,-2.710062,0.255911
9,-0.522154,0.643139,2.547773,7.502978,-5.187312,1.091417


In [25]:
# Now lets get the stats of the tipper data

# Make Array of Max Values
tipperMax = []
for i in range (0,len(tipperLabel)):
    tipperMax.append(tipperDF[(tipperLabel[i])].max())
print ('Tipper Max: ' + str(tipperMax))    

# Make Array of Min Values
tipperMin = []
for i in range (0,len(tipperLabel)):
    tipperMin.append(tipperDF[(tipperLabel[i])].min())
print ('Tipper Min: ' + str(tipperMin))

Tipper Max: [5.27050877, 8.8299427, 89.7246094, 18.7240353, 1.29559386, 44.9219284]
Tipper Min: [-9.29278851, -7.32659435, 0.0586209334, 0.147228658, -8.36588192, 0.0902703479]


In [26]:
#Make Array attributes for tipper parameters
tipAtt = ''
for i in range (0,len(tipperLabel)):
     headerLabel = tipperLabel[i]
     print ('Header Label: ' + headerLabel + '-' + headerLabel[:3])
     if headerLabel.find('R') != -1:
        tipAtti = '\t\t<attr>\n\t\t\t<attrlabl>' + headerLabel + '</attrlabl>\n' \
        + '\t\t\t<attrdef>Imaginary part of complex tipper component ' + headerLabel[:2] + '.</attrdef>' \
        + '\n\t\t\t<attrdefs>U.S. Geological Survey</attrdefs>\n\t\t' \
        + '\n<attrdomv>\n\t\t\t<rdom>\n\t\t\t<rdommin>'+ str(tipperMin[i]) + '</rdommin>\n\t\t\t<rdommax>' + str(tipperMax[i]) + '</rdommax>\n\t\t\t'\
        + '<attrunit>ohm-m</attrunit>\n\t\t\t\t</rdom>\n\t\t\t\t</attrdomv>' \
        + '\t\t</attr>'
        tipAtt = tipAtt + tipAtti
     if headerLabel.find('I') != -1:
        tipAtti = '\t\t<attr>\n\t\t\t<attrlabl>' + headerLabel + '</attrlabl>\n' \
        + '\t\t\t<attrdef>Imaginary component of impedance ' + headerLabel[:2] + ' in ohm-m (Ex / Hx)</attrdef>' \
        + '\n\t\t\t<attrdefs>U.S. Geological Survey</attrdefs>\n\t\t' \
        + '\n<attrdomv>\n\t\t\t<rdom>\n\t\t\t<rdommin>'+ str(tipperMin[i]) + '</rdommin>\n\t\t\t<rdommax>' + str(tipperMax[i]) + '</rdommax>\n\t\t\t'\
        + '<attrunit>ohm-m</attrunit>\n\t\t\t\t</rdom>\n\t\t\t\t</attrdomv>' \
        + '\t\t</attr>'
        tipAtt = tipAtt + tipAtti
     if headerLabel.find('.VAR') != -1:
        tipAtti = '\t\t<attr>\n\t\t\t<attrlabl>' + headerLabel + '</attrlabl>\n' \
        + '\t\t\t<attrdef>Variance of real and imaginary impedance of ' + headerLabel[:2] + ' in ohm-m (Ex / Hx)</attrdef>' \
        + '\n\t\t\t<attrdefs>U.S. Geological Survey</attrdefs>\n\t\t' \
        + '\n<attrdomv>\n\t\t\t<rdom>\n\t\t\t<rdommin>'+ str(tipperMin[i]) + '</rdommin>\n\t\t\t<rdommax>' + str(tipperMax[i]) + '</rdommax>\n\t\t\t'\
        + '<attrunit>ohm-m</attrunit>\n\t\t\t\t</rdom>\n\t\t\t\t</attrdomv>' \
        + '\n\t\t</attr>'
        tipAtt = tipAtt + tipAtti
        
print (tipAtt)
TipperAttributes = tipAtt

Header Label: TXR.EXP-TXR
Header Label: TXI.EXP-TXI
Header Label: TXVAR.EXP-TXV
Header Label: TYR.EXP-TYR
Header Label: TYI.EXP-TYI
Header Label: TYVAR.EXP-TYV
		<attr>
			<attrlabl>TXR.EXP</attrlabl>
			<attrdef>Imaginary part of complex tipper component TX.</attrdef>
			<attrdefs>U.S. Geological Survey</attrdefs>
		
<attrdomv>
			<rdom>
			<rdommin>-9.29278851</rdommin>
			<rdommax>5.27050877</rdommax>
			<attrunit>ohm-m</attrunit>
				</rdom>
				</attrdomv>		</attr>		<attr>
			<attrlabl>TXI.EXP</attrlabl>
			<attrdef>Imaginary component of impedance TX in ohm-m (Ex / Hx)</attrdef>
			<attrdefs>U.S. Geological Survey</attrdefs>
		
<attrdomv>
			<rdom>
			<rdommin>-7.32659435</rdommin>
			<rdommax>8.8299427</rdommax>
			<attrunit>ohm-m</attrunit>
				</rdom>
				</attrdomv>		</attr>		<attr>
			<attrlabl>TXVAR.EXP</attrlabl>
			<attrdef>Imaginary part of complex tipper component TX.</attrdef>
			<attrdefs>U.S. Geological Survey</attrdefs>
		
<attrdomv>
			<rdom>
			<rdommin>0.05862093

Here we load the computed parameters
>!****COMPUTED PARAMETERS****!

In [27]:
# Import entity and attributes - !****COMPUTED PARAMETERS****! plan to break some of these individual chunks into objects/functions
# Probably will need two functions for this - one for a single list and one for the long lists with more than one column
# Get Range of COMPUTED PARAMETERS in EDI File
for k in range(list_length):
 value = list_[k] 
 if value.replace(" ", "") == '>!****COMPUTEDPARAMETERS****!':
   startIndPar = k + 1
   print ('startIndPar: ' + str(startIndPar))
 
 if value.replace(" ", "") ==  '>END':
   endIndPar = k - 1
   print ('endIndPar: ' + str(endIndPar))

#Construct Array of Channel Headers   
count = 0
parLabel = []
parData = []
pdata = []
#Constuct a library of Headers and Values using Pandas, https://pandas.pydata.org/
parDF = pd.DataFrame(pdata)
for l in range(startIndPar,endIndPar): 
    if list_[l][0] == '>':
     ptemp = list_[l].split(" ", 1)
     #print (ptemp)
     parLabel.append((ptemp[0].split(">"))[1])
     pdataTemp = list_[l+1]
     for j in range(l+2,l+8):
      pdataTemp = pdataTemp + list_[j]
      pdataTemp = re.sub(' +', ' ',pdataTemp)
      pdata = pdataTemp.split(" ")
     #print (pdata)
     del pdata[0]
     del pdata[len(pdata)-1] # need to check for empty strings and delete
     pdata = np.array(pdata).astype(np.float) #convert String to floats
     pe = pd.Series(pdata)
     print ((ptemp[0].split(">"))[1]  + ':' + str(l) )
     parDF[((ptemp[0].split(">"))[1])] = te.values
    
    count = count + 1

parDF
#pdata
#pe 

startIndPar: 455
endIndPar: 814
RHOROT:455
RHOXX:464
RHOXX.ERR:473
RHOXY:482
RHOXY.ERR:491
RHOYX:500
RHOYX.ERR:509
RHOYY:518
RHOYY.ERR:527
PHSXX:536
PHSXX.ERR:545
PHSXY:554
PHSXY.ERR:563
PHSYX:572
PHSYX.ERR:581
PHSYY:590
PHSYY.ERR:599
TIPMAG:608
TIPMAG.ERR:617
TIPPHS:626
TIPPHS.ERR:635
ZSTRIKE:644
ZSKEW:653
TSTRIKE:662
COH:671
COH:680
COH:689
COH:698
EPREDCOH:707
EPREDCOH:716
SIGAMP:725
SIGAMP:734
SIGAMP:743
SIGAMP:752
SIGAMP:761
SIGNOISE:770
SIGNOISE:779
SIGNOISE:788
SIGNOISE:797
SIGNOISE:806


,RHOROT,RHOXX,RHOXX.ERR,RHOXY,RHOXY.ERR,RHOYX,RHOYX.ERR,RHOYY,RHOYY.ERR,PHSXX,...,TIPMAG.ERR,TIPPHS,TIPPHS.ERR,ZSTRIKE,ZSKEW,TSTRIKE,COH,EPREDCOH,SIGAMP,SIGNOISE
0,0.597910,0.597910,0.597910,0.597910,0.597910,0.597910,0.597910,0.597910,0.597910,0.597910,...,0.597910,0.597910,0.597910,0.597910,0.597910,0.597910,0.597910,0.597910,0.597910,0.597910
1,0.599377,0.599377,0.599377,0.599377,0.599377,0.599377,0.599377,0.599377,0.599377,0.599377,...,0.599377,0.599377,0.599377,0.599377,0.599377,0.599377,0.599377,0.599377,0.599377,0.599377
2,0.289512,0.289512,0.289512,0.289512,0.289512,0.289512,0.289512,0.289512,0.289512,0.289512,...,0.289512,0.289512,0.289512,0.289512,0.289512,0.289512,0.289512,0.289512,0.289512,0.289512
3,0.285606,0.285606,0.285606,0.285606,0.285606,0.285606,0.285606,0.285606,0.285606,0.285606,...,0.285606,0.285606,0.285606,0.285606,0.285606,0.285606,0.285606,0.285606,0.285606,0.285606
4,1.218923,1.218923,1.218923,1.218923,1.218923,1.218923,1.218923,1.218923,1.218923,1.218923,...,1.218923,1.218923,1.218923,1.218923,1.218923,1.218923,1.218923,1.218923,1.218923,1.218923
5,0.761454,0.761454,0.761454,0.761454,0.761454,0.761454,0.761454,0.761454,0.761454,0.761454,...,0.761454,0.761454,0.761454,0.761454,0.761454,0.761454,0.761454,0.761454,0.761454,0.761454
6,44.921928,44.921928,44.921928,44.921928,44.921928,44.921928,44.921928,44.921928,44.921928,44.921928,...,44.921928,44.921928,44.921928,44.921928,44.921928,44.921928,44.921928,44.921928,44.921928,44.921928
7,1.250830,1.250830,1.250830,1.250830,1.250830,1.250830,1.250830,1.250830,1.250830,1.250830,...,1.250830,1.250830,1.250830,1.250830,1.250830,1.250830,1.250830,1.250830,1.250830,1.250830
8,0.255911,0.255911,0.255911,0.255911,0.255911,0.255911,0.255911,0.255911,0.255911,0.255911,...,0.255911,0.255911,0.255911,0.255911,0.255911,0.255911,0.255911,0.255911,0.255911,0.255911
9,1.091417,1.091417,1.091417,1.091417,1.091417,1.091417,1.091417,1.091417,1.091417,1.091417,...,1.091417,1.091417,1.091417,1.091417,1.091417,1.091417,1.091417,1.091417,1.091417,1.091417


In [28]:
# Now lets get the stats of the computed parameters

# Make Array of Max Values
parMax = []
for i in range (0,len(parLabel)):
    parMax.append(parDF[(parLabel[i])].max())
print ('Computed Pararmeters Max: ' + str(parMax))    

# Make Array of Min Values
parMin = []
for i in range (0,len(parLabel)):
    parMin.append(parDF[(parLabel[i])].min())
print ('Computed Pararmeters Min: ' + str(parMin))

for i in range (0,len(parLabel)):
    print ('Header Index ' + str(parLabel[i]) + ':' + str(i))

Computed Pararmeters Max: [44.9219284, 44.9219284, 44.9219284, 44.9219284, 44.9219284, 44.9219284, 44.9219284, 44.9219284, 44.9219284, 44.9219284, 44.9219284, 44.9219284, 44.9219284, 44.9219284, 44.9219284, 44.9219284, 44.9219284, 44.9219284, 44.9219284, 44.9219284, 44.9219284, 44.9219284, 44.9219284, 44.9219284, 44.9219284, 44.9219284, 44.9219284, 44.9219284, 44.9219284, 44.9219284, 44.9219284, 44.9219284, 44.9219284, 44.9219284, 44.9219284, 44.9219284, 44.9219284, 44.9219284, 44.9219284, 44.9219284]
Computed Pararmeters Min: [0.0902703479, 0.0902703479, 0.0902703479, 0.0902703479, 0.0902703479, 0.0902703479, 0.0902703479, 0.0902703479, 0.0902703479, 0.0902703479, 0.0902703479, 0.0902703479, 0.0902703479, 0.0902703479, 0.0902703479, 0.0902703479, 0.0902703479, 0.0902703479, 0.0902703479, 0.0902703479, 0.0902703479, 0.0902703479, 0.0902703479, 0.0902703479, 0.0902703479, 0.0902703479, 0.0902703479, 0.0902703479, 0.0902703479, 0.0902703479, 0.0902703479, 0.0902703479, 0.0902703479, 0.09

In [29]:
# assign variables for xml file find and replace
RHOROTmax = parMax[0]
RHOROTmin = parMin[0]
RHOXXmax = parMax[1]
RHOXXmin = parMin[1]
PHSXXmax = parMax[9]
PHSXXmin = parMin[9]
RHOXXERRmax = parMax[2]
RHOXXERRmin = parMin[2]
PHSXXERRmax = parMax[10]
PHSXXERRmin = parMin[10]
RHOXYmax = parMax[3]
RHOXYmin = parMin[3]
PHSXYmax = parMax[11]
PHSXYmin = parMin[11]
RHOXYERRmax = parMax[4]
RHOXYERRmin = parMin[4]
PHSXYERRmax = parMax[12]
PHSXYERRmin = parMin[12]
RHOYXmax = parMax[5]
RHOYXmin = parMin[5]
PHSYXmax = parMax[13]
PHSYXmin = parMin[13]
RHOYXERRmax = parMax[6]
RHOYXERRmin = parMin[6]
PHSYXERRmax = parMax[14]
PHSYXERRmin = parMin[14]
RHOYYmax = parMax[7]
RHOYYmin = parMin[7]
PHSYYmax = parMax[15]
PHSYYmin = parMin[15]
RHOYYERRmax = parMax[8]
RHOYYERRmin = parMin[8]
PHSYYERRmax = parMax[16]
PHSYYERRmin = parMin[16]
TIPMAGmax = parMax[17]
TIPMAGmin = parMin[17]
print ('TIPMAGmin = ' + str(TIPMAGmin))
TIPPHSmax = parMax[19]
TIPPHSmin = parMin[19]
TIPMAGERRmax = parMax[18]
TIPMAGERRmin = parMin[18]
TIPPHSERRmax = parMax[20]
TIPPHSERRmin = parMin[20]
ZSTRIKEmax = parMax[21]
ZSTRIKEmin = parMin[21]
ZSKEWmax = parMax[22]
ZSKEWmin = parMin[22]
TSTRIKEmax = parMax[23]
TSTRIKEmin = parMin[23]
COHmax = parMax[24]
COHmin = parMin[24]
EPREDCOHmax = parMax[28]
EPREDCOHmin = parMin[28]
SIGAMPmax = parMax[31]
SIGAMPmin = parMin[31]
SIGNOISEmax = parMax[36]
SIGNOISEmin = parMin[36]

TIPMAGmin = 0.0902703479


## Now lets get the range of values from the RSP values

## Now the raw Binary File Listing - this can be T files or W files
We will need to figure out the best way of filtering on thise - may need to build array and then delete AVG, dmp and edi file.

These are listed in the edi file as well but they are not all there.  

    ProcessingTimeSeriesUsed:
         wp01A1.bp1                                                                     
         wp01A2.bp1                                                                     
         wp01A1.sd6                                                                     
         wp01A2.sd6                                                                     
         wp01A1.sd7                                                                     
         wp01A2.sd8                                                                     
         wp01A2_3.sd9 

- Which files need to be included in the data release?
- What is the best way to get this listing?

In [30]:
#First Get the list of RSP files
rspList = glob.glob(os.path.join(mtStationPath, '*.RSP'),  recursive=True)
#rspList
ediFile = str(ediPathArray[len(ediPathArray)-1])
fileListing = ''
fileListing = fileListing + '\t\t\t\t\t\t' + ediFile + '\n' #start the file listing with the main EDI file
rspFileListing = []
for i in range(len(rspList)):
    splitRspList = rspList[i].split('\\')
    fileListing = fileListing + '\t\t\t\t\t\t' + splitRspList[len(splitRspList) - 1] + '\n'
    rspFileListing.append(splitRspList[len(splitRspList) - 1])
print (fileListing)


						USA-Colorado-Silverton_Caldera-2018-AMTCC01.edi
						BF6-9621.RSP
						BF6-9624.RSP
						BF6-9625.RSP
						EF-9515X.RSP
						EF-9515Y.RSP



In [31]:
#Now finally add the processed ASCII text files to the list
txtList = glob.glob(os.path.join(mtStationPath, '*.txt'),  recursive=True)
txtListFormatted = []
#txtList
print ('Text File Only Listing:\n')
for i in range(len(txtList)):
  splitTxtList = txtList[i].split('\\')
  txtListFormatted.append(splitTxtList[len(splitTxtList) - 1])
  #print (txtListFormatted[i])  
  fileListing = fileListing + '\t\t\t\t\t\t' + splitTxtList[len(splitTxtList) - 1] + '\n'

#print ('File Listing:\n' + fileListing)
#Remove readme file from list.  May need to search an remove if it does not come up first in alphabetical order?'
del txtListFormatted[0]
print (txtListFormatted)

Text File Only Listing:

['USA-Colorado-Silverton_Caldera-2018-AMTCC01-BP_High_01.txt', 'USA-Colorado-Silverton_Caldera-2018-AMTCC01-BP_High_02.txt', 'USA-Colorado-Silverton_Caldera-2018-AMTCC01-FC6_01.txt', 'USA-Colorado-Silverton_Caldera-2018-AMTCC01-FC6_02.txt', 'USA-Colorado-Silverton_Caldera-2018-AMTCC01-FC7_01.txt', 'USA-Colorado-Silverton_Caldera-2018-AMTCC01-FC7_02.txt', 'USA-Colorado-Silverton_Caldera-2018-AMTCC01-FC8_01.txt', 'USA-Colorado-Silverton_Caldera-2018-AMTCC01-FC8_02.txt', 'USA-Colorado-Silverton_Caldera-2018-AMTCC01-FC9_01.txt', 'USA-Colorado-Silverton_Caldera-2018-AMTCC01-FC9_02.txt', 'USA-Colorado-Silverton_Caldera-2018-AMTCC01-SD6_01.txt', 'USA-Colorado-Silverton_Caldera-2018-AMTCC01-SD6_02.txt', 'USA-Colorado-Silverton_Caldera-2018-AMTCC01-SD7_01.txt', 'USA-Colorado-Silverton_Caldera-2018-AMTCC01-SD7_02.txt', 'USA-Colorado-Silverton_Caldera-2018-AMTCC01-SD8_01.txt', 'USA-Colorado-Silverton_Caldera-2018-AMTCC01-SD8_02.txt', 'USA-Colorado-Silverton_Caldera-2018-A

In [32]:
# Create the Entity and Attributes for the .txt? listing
txtEandA = ''
#As before, we read in the list of files

for i in range(len(txtListFormatted)):
    if txtListFormatted[i].find('-BP') != -1:
        txtEandA = txtEandA + '\t\t<detailed>\n\t\t\t<enttyp>\n\t\t\t\t<enttypl>Text File ' + txtListFormatted[i] + '</enttypl>\n' \
            + '\t\t\t\t<enttypd>Header file in ASCII text format for raw cross power files</enttypd>' \
            + '\n\t\t\t\t<enttypds>U.S. Geological Survey</enttypds>\n\t\t\t</enttyp>' \
            + '\n\t\t\t<attr>\n\t\t\t\t<attrlabl>Header Information</attrlabl>\n\t\t\t\t<attrdef>Header description and settings for cross power content</attrdef>\n\t\t\t\t<attrdefs>U.S. Geological Survey</attrdefs>\n\t\t\t\t<attrdomv>'\
            + '\n\t\t\t\t\t<udom>Header description and settings for cross power content</udom>\n\t\t\t\t</attrdomv>' \
            + '\n\t\t\t</attr>\n\t\t</detailed>\n' 
    if txtListFormatted[i].find('-FC') != -1:
        txtEandA = txtEandA + '\t\t<detailed>\n\t\t\t<enttyp>\n\t\t\t\t<enttypl>Text File ' + txtListFormatted[i] + '</enttypl>\n' \
            + '\t\t\t\t<enttypd>Header file in ASCII text format for raw fourier coefficient files</enttypd>' \
            + '\n\t\t\t\t<enttypds>U.S. Geological Survey</enttypds>\n\t\t\t</enttyp>' \
            + '\n\t\t\t<attr>\n\t\t\t\t<attrlabl>Header Information</attrlabl>\n\t\t\t\t<attrdef>Header description and settings for fourier coefficient binary content</attrdef>\n\t\t\t\t<attrdefs>U.S. Geological Survey</attrdefs>\n\t\t\t\t<attrdomv>'\
            + '\n\t\t\t\t\t<udom>Header description and settings for fourier coefficient binary content</udom>\n\t\t\t\t</attrdomv>' \
            + '\n\t\t\t</attr>\n\t\t</detailed>\n'
    if txtListFormatted[i].find('-SD') != -1:
        txtEandA = txtEandA + '\t\t<detailed>\n\t\t\t<enttyp>\n\t\t\t\t<enttypl>Text File ' + txtListFormatted[i] + '</enttypl>\n' \
            + '\t\t\t\t<enttypd>Header file in ASCII text format for sychronous detection cross power files</enttypd>' \
            + '\n\t\t\t\t<enttypds>U.S. Geological Survey</enttypds>\n\t\t\t</enttyp>' \
            + '\n\t\t\t<attr>\n\t\t\t\t<attrlabl>Header Information</attrlabl>\n\t\t\t\t<attrdef>Header description and settings for cross power content</attrdef>\n\t\t\t\t<attrdefs>U.S. Geological Survey</attrdefs>\n\t\t\t\t<attrdomv>'\
            + '\n\t\t\t\t\t<udom>Header description and settings for cross power content</udom>\n\t\t\t\t</attrdomv>' \
            + '\n\t\t\t</attr>\n\t\t</detailed>\n'
            
    if txtListFormatted[i].find('-TS') != -1:
        txtEandA = txtEandA + '\t\t<detailed>\n\t\t\t<enttyp>\n\t\t\t\t<enttypl>Text File ' + txtListFormatted[i] + '</enttypl>\n' \
            + '\t\t\t\t<enttypd>Header file in ASCII text format for binary time series files</enttypd>' \
            + '\n\t\t\t\t<enttypds>U.S. Geological Survey</enttypds>\n\t\t\t</enttyp>' \
            + '\n\t\t\t<attr>\n\t\t\t\t<attrlabl>Header Information</attrlabl>\n\t\t\t\t<attrdef>Header description and settings for time series binary content</attrdef>\n\t\t\t\t<attrdefs>U.S. Geological Survey</attrdefs>\n'\
            + '\t\t\t\t<attrdomv>\n\t\t\t\t\t<udom>Header description and settings for time series binary content</udom>\n\t\t\t\t</attrdomv>' \
            + '\n\t\t\t</attr>\n\t\t</detailed>\n'
    
#Add the final .txt listing for the readme file.  
#Be sure to comment this out if there is no readme.txt file included with the data release

readmeEandA = '\t\t<detailed>\n\t\t<enttyp>\n\t\t\t<enttypl>Text File readme.txt</enttypl>'\
        + '\n\t\t\t<enttypd>Read Me file describing the naming format of the EDI files and that they may need to be renamed to be imported into certain software packages.</enttypd>'\
        + '\n\t\t\t<enttypds>U.S. Geological Survey</enttypds>'\
        + '\n\t\t</enttyp>\n\t\t</detailed>\n'
    
txtEandA = txtEandA + readmeEandA
print (txtEandA)
ALL_EandA = '' #'<eainfo>\n'   
ALL_EandA = ALL_EandA + txtEandA

		<detailed>
			<enttyp>
				<enttypl>Text File USA-Colorado-Silverton_Caldera-2018-AMTCC01-BP_High_01.txt</enttypl>
				<enttypd>Header file in ASCII text format for raw cross power files</enttypd>
				<enttypds>U.S. Geological Survey</enttypds>
			</enttyp>
			<attr>
				<attrlabl>Header Information</attrlabl>
				<attrdef>Header description and settings for cross power content</attrdef>
				<attrdefs>U.S. Geological Survey</attrdefs>
				<attrdomv>
					<udom>Header description and settings for cross power content</udom>
				</attrdomv>
			</attr>
		</detailed>
		<detailed>
			<enttyp>
				<enttypl>Text File USA-Colorado-Silverton_Caldera-2018-AMTCC01-BP_High_02.txt</enttypl>
				<enttypd>Header file in ASCII text format for raw cross power files</enttypd>
				<enttypds>U.S. Geological Survey</enttypds>
			</enttyp>
			<attr>
				<attrlabl>Header Information</attrlabl>
				<attrdef>Header description and settings for cross power content</attrdef>
				<attrdefs>U.S. Geological Survey

# Now get values and stats on the .RSP files that are listed to add to the Ent. and Att. information

### These files are all fixed width format (666) but it looks like the BFS*.RSP and EF*.RSP are slightly different beasts that have different formats

In [33]:
# Load the BFS*.RSP files into pandasand create entities and attributes
allRspEandP = ''
strFreqMaxBSP = ''
strAmpMaxBSP = ''
strGammaMaxBSP = ''
strFreqMinBSP = ''
strAmpMinBSP = ''
strGammaMinBSP = ''

strFreqMaxESP = ''
strFreqMinESP = ''
strAmp1MaxESP = ''
strAmp1MinESP = ''
strAmp2MaxESP = ''
strAmp2MinESP = ''
strAmp3MaxESP = ''
strAmp3MinESP = ''
strAmp4MaxESP = ''
strAmp4MinESP = ''
strPhz1MaxESP = ''
strPhz1MinESP = ''
strPhz2MaxESP = ''
strPhz2MinESP = ''
strPhz3MaxESP = ''
strPhz3MinESP = ''
strPhz4MaxESP = ''
strPhz4MinESP = ''

intNumberBSF = 0
intNumberEFF = 0
for i in range(len(rspList)):
#for i in range(3):
    if rspList[i].find('BF') > 0: #Do this if the file has a BF style Fixed Width Format
        bfRSP = pd.read_fwf(rspList[i], widths=[6,6,6], skiprows=5, parse_dates=True).rename(columns={'31':'Freq', '1':'Amp', 'Unnamed: 2':'Gamma'})
#print (rspList[1])
        #print (bfRSP)    
    
# Now lets get the stats of the bfRSP data

# Make Array of Min and Max Values
        FreqMaxRSP = bfRSP['Freq'].max()
        strFreqMaxBSP = str(FreqMaxRSP)
        AmpMaxRSP = bfRSP['Amp'].max()
        strAmpMaxBSP = str(AmpMaxRSP)
        GammaMaxRSP = bfRSP['Gamma'].max()
        strGammaMaxBSP = str(GammaMaxRSP)
        FreqMinRSP = bfRSP['Freq'].min()
        strFreqMinBSP = str(FreqMinRSP)
        AmpMinRSP = bfRSP['Amp'].min()
        strAmpMinBSP = str(AmpMinRSP)
        GammaMinRSP = bfRSP['Gamma'].min()
        strGammaMinBSP = str(GammaMinRSP)
# now print RSP entity and attribute    
        bspEandA = '\t\t<detailed>\n\t\t<enttyp>\n\t\t\t<enttypl>Text File ' + rspFileListing [i] + '</enttypl>\n' \
        + '\t\t\t<enttypd>System Calibration File</enttypd>\n\t\t\t<enttypds>Electromagnetic Instruments (EMI)</enttypds>\n\t\t</enttyp>\n' \
        + '\t\t\t<attr>\n\t\t\t<attrlabl>Freq</attrlabl>\n\t\t\t<attrdef>Frequency - Hz</attrdef>\n\t\t\t<attrdefs>U.S. Geological Survey</attrdefs>' \
        + '\n\t\t\t<attrdomv>\n\t\t\t\t<rdom>\n\t\t\t\t<rdommin>' \
        + strFreqMinBSP + '</rdommin>\n\t\t\t\t<rdommax>' + strFreqMaxBSP + '</rdommax>\n\t\t\t\t' \
        + '<attrunit>Hz</attrunit>\n\t\t\t</rdom>\n\t\t\t</attrdomv>\n\t\t</attr>\n' \
        + '\t\t\t<attr>\n\t\t\t<attrlabl>Amp</attrlabl>\n\t\t\t<attrdef>Amplitude - Volts/Gamma</attrdef>\n\t\t\t<attrdefs>U.S. Geological Survey</attrdefs>' \
        + '\n\t\t\t<attrdomv>\n\t\t\t\t<rdom>\n\t\t\t\t<rdommin>' \
        + strAmpMinBSP + '</rdommin>\n\t\t\t\t<rdommax>' + strAmpMaxBSP + '</rdommax>\n\t\t\t\t' \
        + '<attrunit>Volts/Gamma</attrunit>\n\t\t\t</rdom>\n\t\t\t</attrdomv>\n\t\t</attr>\n' \
        + '\t\t\t<attr>\n\t\t\t<attrlabl>Phz</attrlabl>\n\t\t\t<attrdef>Phase - Degrees</attrdef>\n\t\t\t<attrdefs>U.S. Geological Survey</attrdefs>' \
        + '\n\t\t\t<attrdomv>\n\t\t\t\t<rdom>\n\t\t\t\t<rdommin>' \
        + strGammaMinBSP + '</rdommin>\n\t\t\t\t<rdommax>' + strGammaMaxBSP + '</rdommax>\n\t\t\t\t' \
        + '<attrunit>Degrees</attrunit>\n\t\t\t</rdom>\n\t\t\t</attrdomv>\n\t\t</attr>\n' \
        + '\t</detailed>'
        print ('i Loop = ' + str(i))
        allRspEandP = allRspEandP +  bspEandA + '\n'
    if rspList[i].find('EF') > 0:
        print ('i Loop = ' + str(i) + ' EF File Found')
        efRSP = pd.read_fwf(rspList[i], widths=[6,6,6,6,6,6,6,6,6], skiprows=5, parse_dates=True)\
        .rename(columns={'42':'Freq', '4':'Amp1', 'Low F':'Phz1', 'requen':'Amp2', 'cy':'Phz2', '10Hz':'Amp3', 'Out':'Phz3',\
        'Unnamed: 7':'Amp4', 'Unnamed: 8':'Phz4'})
#Delete headers from data to remove all strings from Pandas Data Frame

        efRSP = efRSP[efRSP.Amp2 != 'Freque']

# Make Array of Min and Max Values
        strFreqMaxESP = str(efRSP['Freq'].max())
        strFreqMinESP = str(efRSP['Freq'].min())
        strAmp1MaxESP = str(efRSP['Amp1'].max())
        strAmp1MinESP = str(efRSP['Amp1'].min())
        strAmp2MaxESP = str(efRSP['Amp2'].max())
        strAmp2MinESP = str(efRSP['Amp2'].min())
        strAmp3MaxESP = str(efRSP['Amp3'].max())
        strAmp3MinESP = str(efRSP['Amp3'].min())
        strAmp4MaxESP = str(efRSP['Amp4'].max())
        strAmp4MinESP = str(efRSP['Amp4'].min())
        strPhz1MaxESP = str(efRSP['Phz1'].max())
        strPhz1MinESP = str(efRSP['Phz1'].min())
        strPhz2MaxESP = str(efRSP['Phz2'].max())
        strPhz2MinESP = str(efRSP['Phz2'].min())
        strPhz3MaxESP = str(efRSP['Phz3'].max())
        strPhz3MinESP = str(efRSP['Phz3'].min())
        strPhz4MaxESP = str(efRSP['Phz4'].max())
        strPhz4MinESP = str(efRSP['Phz4'].min())
        espEandA = '\t<detailed>\n\t\t\t<enttyp>\n\t\t\t<enttypl>Text File ' + rspFileListing [i] + '</enttypl>\n' \
        + '\t\t\t<enttypd>System Calibration File</enttypd>\n\t\t\t<enttypds>Electromagnetic Instruments (EMI)</enttypds>\n\t\t</enttyp>\n' \
        + '\t\t\t<attr>\n\t\t\t<attrlabl>Freq</attrlabl>\n\t\t\t<attrdef>Frequency - Hz</attrdef>\n\t\t\t<attrdefs>U.S. Geological Survey</attrdefs>' \
        + '\n\t\t\t<attrdomv>\n\t\t\t\t<rdom>\n\t\t\t\t<rdommin>' \
        + strFreqMinESP + '</rdommin>\n\t\t\t\t<rdommax>' + strFreqMaxESP + '</rdommax>\n\t\t\t\t' \
        + '<attrunit>Hz</attrunit>\n\t\t\t</rdom>\n\t\t\t</attrdomv>\n\t\t</attr>\n' \
        + '\t\t<attr>\n\t\t\t<attrlabl>Amp</attrlabl>\n\t\t\t<attrdef>Amplitude - Volts</attrdef>\n\t\t\t<attrdefs>U.S. Geological Survey</attrdefs>' \
        + '\n\t\t\t<attrdomv>\n\t\t\t\t<rdom>\n\t\t\t\t<rdommin>' \
        + strAmp1MinESP + '</rdommin>\n\t\t\t\t<rdommax>' + strAmp1MaxESP + '</rdommax>\n\t\t\t\t' \
        + '<attrunit>Volts</attrunit>\n\t\t\t</rdom>\n\t\t\t</attrdomv>\n\t\t</attr>\n' \
        + '\t\t\t<attr>\n\t\t\t<attrlabl>Phz</attrlabl>\n\t\t\t<attrdef>Phase - Degrees</attrdef>\n\t\t\t<attrdefs>U.S. Geological Survey</attrdefs>' \
        + '\n\t\t\t<attrdomv>\n\t\t\t\t<rdom>\n\t\t\t\t<rdommin>' \
        + strPhz1MinESP + '</rdommin>\n\t\t\t\t<rdommax>' + strPhz1MaxESP + '</rdommax>\n\t\t\t\t' \
        + '<attrunit>Degrees</attrunit>\n\t\t\t</rdom>\n\t\t\t</attrdomv>\n\t\t</attr>\n' \
        + '\t</detailed>'
        print ('i Loop = ' + str(i))
        allRspEandP = allRspEandP +  espEandA + '\n' 
        print (str(allRspEandP)) 

        
EandA = allRspEandP # [-:2] # this removes the last characters of the string to get rid of the last line return.  
ALL_EandA = ALL_EandA +  allRspEandP


i Loop = 0
i Loop = 1
i Loop = 2
i Loop = 3 EF File Found
i Loop = 3
		<detailed>
		<enttyp>
			<enttypl>Text File BF6-9621.RSP</enttypl>
			<enttypd>System Calibration File</enttypd>
			<enttypds>Electromagnetic Instruments (EMI)</enttypds>
		</enttyp>
			<attr>
			<attrlabl>Freq</attrlabl>
			<attrdef>Frequency - Hz</attrdef>
			<attrdefs>U.S. Geological Survey</attrdefs>
			<attrdomv>
				<rdom>
				<rdommin>0.1</rdommin>
				<rdommax>25000.0</rdommax>
				<attrunit>Hz</attrunit>
			</rdom>
			</attrdomv>
		</attr>
			<attr>
			<attrlabl>Amp</attrlabl>
			<attrdef>Amplitude - Volts/Gamma</attrdef>
			<attrdefs>U.S. Geological Survey</attrdefs>
			<attrdomv>
				<rdom>
				<rdommin>0.0019100000000000002</rdommin>
				<rdommax>0.16940999999999998</rdommax>
				<attrunit>Volts/Gamma</attrunit>
			</rdom>
			</attrdomv>
		</attr>
			<attr>
			<attrlabl>Phz</attrlabl>
			<attrdef>Phase - Degrees</attrdef>
			<attrdefs>U.S. Geological Survey</attrdefs>
			<attrdomv>
				<rdom>
				<rdommi

## Now let's work on the SD mode files

### Overview of SD files,

The prefix file naming convention for the SD mode AMT non-transmitter files are:
AANNNRN where AA is the survey area, NNN is the site number, R is the run "number" (run A, run B, run C, etc.), and N is a subset run number (first run is A1, second run is A2, third run is A3, etc.)

The suffix file naming convention for the SD mode AMT non-transmitter files are:
SD6 sample frequencies are always 79, 90, 100, 150, 210, 270, 340, 460, 580 Hertz
SD7 sample frequencies are always 340, 460, 580, 720, 885, 1170 Hertz
SD8 sample frequencies are always 1170, 1500, 1870, 2200, 2730, 3550, 4900, 6500, 9000 Hertz
SD9 sample frequencies are always 6500, 9000, 11590, 15290, 19500, 23370 Hertz

The prefix file naming convention for the SD mode AMT transmitter files are:
AANNNRN where AA is the survey area, NNN is the site number, R is the run "number" (run A, run B, run C, etc.), and N is a subset run "number" (first run is AA, second run is AB, third run is AC, etc.)

The suffix file naming convention for the SD mode AMT transmitter files are:
SDA sample frequency is always 960 Hertz
SDB sample frequency is always 1200 Hertz
SDC sample frequency is always 1870 Hertz
SDD sample frequency is always 2420 Hertz
SDE sample frequency is always 2730 Hertz
SDF sample frequency is always 3550 Hertz
SDG sample frequency is always 5210 Hertz
SDH sample frequency is always 6850 Hertz
SDI sample frequency is always 11590 Hertz
SDJ sample frequency is always 15920 Hertz
SDK sample frequency is always 23370 Hertz

The file format of the SD mode files are described in each of the text files (last block of information within each file). It is the same data format regardless of whether or not a transmitter was used to collect the data. We must have been looking at the text file for the FC (Fourier Coefficient) files in your office, so I was confused that I hadn't described the data format (it is there in the text files for all SD files).



In [34]:
#First get a listing of the SD mode files
#Now finally add the processed ASCII text files to the list
sdList = glob.glob(os.path.join(mtStationPath, '*.sd?'),  recursive=True)
#sdaList
sdFileList = ''
for i in range(len(sdList)):
  splitSdList = sdList[i].split('\\')
  fileListing = fileListing + '\t\t\t\t\t\t' + splitSdList[len(splitSdList) - 1] + '\n'
  sdFileList = sdFileList + splitSdList[len(splitSdList) - 1] + '\n'

#print ('File ListingfileListing:\n' + fileListing)
print ('SD file listing:\n' + sdFileList)


SD file listing:
CC01A1.SD6
CC01A1.SD7
CC01A1.SD8
CC01A1.SD9
CC01A2.SD6
CC01A2.SD7
CC01A2.SD8
CC01A2.SD9



In [35]:
#As before, we read in the list of files

for i in range(len(sdList)):
#for i in range(3):
    #if rspList[i].find('BF') > 0: #Do this if the file has a BF style Fixed Width Format
        dfSD = pd.read_fwf(sdList[i], widths=[15,15,15,15,15], skiprows=27, parse_dates=True).rename(columns={'79.0000  32.':'Amp1', '00000   20   1':'Amp2', 'Unnamed: 2':'Amp3', 'Unnamed: 3':'Amp4', 'Unnamed: 4':'Amp5'})
#Now strip out the header rows in the data as to not screw up calculating the range
        #dfSD = dfSD[dfSD.Amp4 != '00000   20   1']
        print (dfSD)        
    

               Amp1             Amp2          Amp3       Amp4       Amp5
0   92.21826113e-01              0.0 -3.402838e+00  -0.986573   1.798557
1               0.0  -3.27370312e-03  1.362538e-03   0.001675  -0.001439
2   33.34272405e-07              0.0 -3.522130e-03   0.003593   0.000752
3   -1.51025504e-03  12.26214711e-07 -8.473739e-07   0.000003   0.000000
4   71.32823624e-03  -4.37455806e-02 -2.404797e-02   0.027240  -0.000040
5   53.02807981e-07  -3.92039958e-05 -1.115690e-05   0.000857   0.000000
6      90.0000  32.   00000   20   1           NaN        NaN        NaN
7   85.61246422e-01              0.0 -1.636013e+00   0.443962   4.633386
8               0.0  31.29559563e-05  2.410386e-04   0.003700  -0.000468
9   57.37039654e-07              0.0 -5.639279e-03   0.002615   0.000186
10  -1.71768598e-04  -1.00767578e-06 -3.910846e-08   0.000007   0.000000
11  36.27846233e-03  -2.87299746e-02 -1.533723e-02   0.008892  -0.000010
12  -1.80847037e-06  -2.44895254e-05  7.354711e-06 

In [36]:
#listing of the Fourier Coefficient files (FC) files.
#These are Binary so we don't need to come up with the range of the values.
fcList = glob.glob(os.path.join(mtStationPath, '*.fc?'),  recursive=True)
#fcList
for i in range(len(fcList)):
  splitFcList = fcList[i].split('\\')
  fileListing = fileListing + '\t\t\t\t\t\t' + splitFcList[len(splitFcList) - 1] + '\n'

print ('File ListingfileListing:\n' + fileListing)

File ListingfileListing:
						USA-Colorado-Silverton_Caldera-2018-AMTCC01.edi
						BF6-9621.RSP
						BF6-9624.RSP
						BF6-9625.RSP
						EF-9515X.RSP
						EF-9515Y.RSP
						readme.txt
						USA-Colorado-Silverton_Caldera-2018-AMTCC01-BP_High_01.txt
						USA-Colorado-Silverton_Caldera-2018-AMTCC01-BP_High_02.txt
						USA-Colorado-Silverton_Caldera-2018-AMTCC01-FC6_01.txt
						USA-Colorado-Silverton_Caldera-2018-AMTCC01-FC6_02.txt
						USA-Colorado-Silverton_Caldera-2018-AMTCC01-FC7_01.txt
						USA-Colorado-Silverton_Caldera-2018-AMTCC01-FC7_02.txt
						USA-Colorado-Silverton_Caldera-2018-AMTCC01-FC8_01.txt
						USA-Colorado-Silverton_Caldera-2018-AMTCC01-FC8_02.txt
						USA-Colorado-Silverton_Caldera-2018-AMTCC01-FC9_01.txt
						USA-Colorado-Silverton_Caldera-2018-AMTCC01-FC9_02.txt
						USA-Colorado-Silverton_Caldera-2018-AMTCC01-SD6_01.txt
						USA-Colorado-Silverton_Caldera-2018-AMTCC01-SD6_02.txt
						USA-Colorado-Silverton_Caldera-2018-AMTCC01-SD7_01.txt
						US

In [37]:
# Create the Entity and Attributes for the FC listing
fcEandAi = ''
fcEandA = ''
for i in range(len(fcList)):
    splitFcList = fcList[i].split('\\')
    fcEandAi = '\t\t<detailed>\n\t\t<enttyp>\n\t\t\t<enttypl>Binary File ' + splitFcList[len(splitFcList) - 1] + '</enttypl>\n' \
        + '\t\t\t<enttypd>Raw Fourier Coefficient Binary Data File. The header information for this file contains field notes including survey identifier, personnel, date/time, weather, indication of a remote MT site, additional comments and sensor configuration for the electric and magnetic field channels.</enttypd>' \
        + '\n\t\t\t<enttypds>U.S. Geological Survey</enttypds>\n\t\t</enttyp>' \
        + '\n<attr>\n\t\t\t<attrlabl>Data Value</attrlabl>\n\t\t\t<attrdef>Binary Data Value</attrdef>\n\t\t\t<attrdefs>U.S. Geological Survey</attrdefs>\n'\
        + '\t\t\t<attrdomv>\n\t\t\t\t<udom>Binary Data Value</udom>\n\t\t\t\t</attrdomv>' \
        + '\t\t</attr>\n\t</detailed>\n'
    fcEandA = fcEandA + fcEandAi

ALL_EandA = ALL_EandA + fcEandA    
print (fcEandA)        
        
        
        
        

		<detailed>
		<enttyp>
			<enttypl>Binary File CC01A1.FC6</enttypl>
			<enttypd>Raw Fourier Coefficient Binary Data File. The header information for this file contains field notes including survey identifier, personnel, date/time, weather, indication of a remote MT site, additional comments and sensor configuration for the electric and magnetic field channels.</enttypd>
			<enttypds>U.S. Geological Survey</enttypds>
		</enttyp>
<attr>
			<attrlabl>Data Value</attrlabl>
			<attrdef>Binary Data Value</attrdef>
			<attrdefs>U.S. Geological Survey</attrdefs>
			<attrdomv>
				<udom>Binary Data Value</udom>
				</attrdomv>		</attr>
	</detailed>
		<detailed>
		<enttyp>
			<enttypl>Binary File CC01A1.FC7</enttypl>
			<enttypd>Raw Fourier Coefficient Binary Data File. The header information for this file contains field notes including survey identifier, personnel, date/time, weather, indication of a remote MT site, additional comments and sensor configuration for the electric and magnetic fie

In [38]:
#listing of the Time Series TS? files
#Now finally add the processed ASCII text files to the list
tsList = glob.glob(os.path.join(mtStationPath, '*.ts?'),  recursive=True)
#tsList
for i in range(len(tsList)):
  splitTsList = tsList[i].split('\\')
  fileListing = fileListing + '\t\t\t\t\t\t' + splitTsList[len(splitTsList) - 1] + '\n'

print ('File ListingfileListing:\n' + fileListing)

File ListingfileListing:
						USA-Colorado-Silverton_Caldera-2018-AMTCC01.edi
						BF6-9621.RSP
						BF6-9624.RSP
						BF6-9625.RSP
						EF-9515X.RSP
						EF-9515Y.RSP
						readme.txt
						USA-Colorado-Silverton_Caldera-2018-AMTCC01-BP_High_01.txt
						USA-Colorado-Silverton_Caldera-2018-AMTCC01-BP_High_02.txt
						USA-Colorado-Silverton_Caldera-2018-AMTCC01-FC6_01.txt
						USA-Colorado-Silverton_Caldera-2018-AMTCC01-FC6_02.txt
						USA-Colorado-Silverton_Caldera-2018-AMTCC01-FC7_01.txt
						USA-Colorado-Silverton_Caldera-2018-AMTCC01-FC7_02.txt
						USA-Colorado-Silverton_Caldera-2018-AMTCC01-FC8_01.txt
						USA-Colorado-Silverton_Caldera-2018-AMTCC01-FC8_02.txt
						USA-Colorado-Silverton_Caldera-2018-AMTCC01-FC9_01.txt
						USA-Colorado-Silverton_Caldera-2018-AMTCC01-FC9_02.txt
						USA-Colorado-Silverton_Caldera-2018-AMTCC01-SD6_01.txt
						USA-Colorado-Silverton_Caldera-2018-AMTCC01-SD6_02.txt
						USA-Colorado-Silverton_Caldera-2018-AMTCC01-SD7_01.txt
						US

In [39]:
# Create the Entity and Attributes for the Time series .TS? listing

tsEandAi = ''
tsEandA = ''
for i in range(len(tsList)):
    splitTsList = tsList[i].split('\\')
    tsEandAi = '\t\t<detailed>\n\t\t<enttyp>\n\t\t\t<enttypl>Binary File ' + splitTsList[len(splitTsList) - 1] + '</enttypl>\n' \
        + '\t\t\t<enttypd>Raw Time Series Binary Data File. The header information for this file contains field notes including survey identifier, personnel, date/time, weather, indication of a remote MT site, additional comments and sensor configuration for the electric and magnetic field channels.</enttypd>' \
        + '\n\t\t\t<enttypds>U.S. Geological Survey</enttypds>' \
        + '\n\t\t</enttyp>\n\t\t<attr>\n\t\t\t<attrlabl>Data Value</attrlabl>\n\t\t\t<attrdef>Binary Data Value</attrdef>\n\t\t\t<attrdefs>U.S. Geological Survey</attrdefs>\n'\
        + '\t\t\t<attrdomv>\n\t\t\t\t<udom>Binary Data Value</udom>\n\t\t\t\t</attrdomv>' \
        + '\t\t</attr>\n\t</detailed>\n'
    tsEandA = tsEandA + tsEandAi

ALL_EandA = ALL_EandA + tsEandA
print (tsEandA)

		<detailed>
		<enttyp>
			<enttypl>Binary File CC01A1.TS1</enttypl>
			<enttypd>Raw Time Series Binary Data File. The header information for this file contains field notes including survey identifier, personnel, date/time, weather, indication of a remote MT site, additional comments and sensor configuration for the electric and magnetic field channels.</enttypd>
			<enttypds>U.S. Geological Survey</enttypds>
		</enttyp>
		<attr>
			<attrlabl>Data Value</attrlabl>
			<attrdef>Binary Data Value</attrdef>
			<attrdefs>U.S. Geological Survey</attrdefs>
			<attrdomv>
				<udom>Binary Data Value</udom>
				</attrdomv>		</attr>
	</detailed>
		<detailed>
		<enttyp>
			<enttypl>Binary File CC01A2.TS1</enttypl>
			<enttypd>Raw Time Series Binary Data File. The header information for this file contains field notes including survey identifier, personnel, date/time, weather, indication of a remote MT site, additional comments and sensor configuration for the electric and magnetic field channels.</

In [40]:
#listing of the BP? files
#Now finally add the processed ASCII text files to the list
bpList = glob.glob(os.path.join(mtStationPath, '*.bp?'),  recursive=True)
#bpList
for i in range(len(bpList)):
  splitBpList = bpList[i].split('\\')
  fileListing = fileListing + '\t\t\t\t\t\t' + splitBpList[len(splitBpList) - 1] + '\n'

print ('File ListingfileListing:\n' + fileListing)

File ListingfileListing:
						USA-Colorado-Silverton_Caldera-2018-AMTCC01.edi
						BF6-9621.RSP
						BF6-9624.RSP
						BF6-9625.RSP
						EF-9515X.RSP
						EF-9515Y.RSP
						readme.txt
						USA-Colorado-Silverton_Caldera-2018-AMTCC01-BP_High_01.txt
						USA-Colorado-Silverton_Caldera-2018-AMTCC01-BP_High_02.txt
						USA-Colorado-Silverton_Caldera-2018-AMTCC01-FC6_01.txt
						USA-Colorado-Silverton_Caldera-2018-AMTCC01-FC6_02.txt
						USA-Colorado-Silverton_Caldera-2018-AMTCC01-FC7_01.txt
						USA-Colorado-Silverton_Caldera-2018-AMTCC01-FC7_02.txt
						USA-Colorado-Silverton_Caldera-2018-AMTCC01-FC8_01.txt
						USA-Colorado-Silverton_Caldera-2018-AMTCC01-FC8_02.txt
						USA-Colorado-Silverton_Caldera-2018-AMTCC01-FC9_01.txt
						USA-Colorado-Silverton_Caldera-2018-AMTCC01-FC9_02.txt
						USA-Colorado-Silverton_Caldera-2018-AMTCC01-SD6_01.txt
						USA-Colorado-Silverton_Caldera-2018-AMTCC01-SD6_02.txt
						USA-Colorado-Silverton_Caldera-2018-AMTCC01-SD7_01.txt
						US

In [41]:
# Create the Entity and Attributes for the .BP? listing

#As before, we read in the list of files

print (bpList)


for i in range(len(bpList)):
#for i in range(3):
    #if rspList[i].find('BF') > 0: #Do this if the file has a BF style Fixed Width Format
        dfBP = pd.read_fwf(bpList[i], widths=[15,15,15,15,15], skiprows=27, parse_dates=True).rename(columns={'4.3945   1.':'Amp1', '85697    1   2':'Amp2', 'Unnamed: 2':'Amp3', 'Unnamed: 3':'Amp4', 'Unnamed: 4':'Amp5'})
#Now strip out the header rows in the data as to not screw up calculating the range
        #dfSD = dfSD[dfSD.Amp4 != '00000   20   1']
        #print (dfBP)        

bpEandAi = ''
bpEandA = ''
for i in range(len(bpList)):
    splitBpList = bpList[i].split('\\')
    bpEandAi = '\t\t<detailed>\n\t\t<enttyp>\n\t\t\t<enttypl>Text File ' + splitBpList[len(splitBpList) - 1] + '</enttypl>\n' \
        + '\t\t\t<enttypd>Raw Cross Power Data File. The header information for this file contains field notes including survey identifier, personnel, date/time, weather, indication of a remote MT site, additional comments and sensor configuration for the electric and magnetic field channels.</enttypd>' \
        + '\n\t\t\t<enttypds>U.S. Geological Survey</enttypds>\n\t\t</enttyp>' \
        + '\n<attr>\n\t\t\t<attrlabl>Data Value</attrlabl>\n\t\t\t<attrdef>Data Value</attrdef>\n\t\t\t<attrdefs>U.S. Geological Survey</attrdefs>\n\t\t\t<attrdomv>'\
        + '\n\t\t\t\t<udom>Data Value</udom>\n\t\t\t\t</attrdomv>' \
        + '\t\t</attr>\n\t</detailed>\n'
    bpEandA = bpEandA + bpEandAi
    
ALL_EandA = ALL_EandA +  bpEandA
print (bpEandA)

['C:\\CurrentWork\\DataReleases\\AMT_Silverton\\CC01\\CC01A1.BP1', 'C:\\CurrentWork\\DataReleases\\AMT_Silverton\\CC01\\CC01A2.BP1']
		<detailed>
		<enttyp>
			<enttypl>Text File CC01A1.BP1</enttypl>
			<enttypd>Raw Cross Power Data File. The header information for this file contains field notes including survey identifier, personnel, date/time, weather, indication of a remote MT site, additional comments and sensor configuration for the electric and magnetic field channels.</enttypd>
			<enttypds>U.S. Geological Survey</enttypds>
		</enttyp>
<attr>
			<attrlabl>Data Value</attrlabl>
			<attrdef>Data Value</attrdef>
			<attrdefs>U.S. Geological Survey</attrdefs>
			<attrdomv>
				<udom>Data Value</udom>
				</attrdomv>		</attr>
	</detailed>
		<detailed>
		<enttyp>
			<enttypl>Text File CC01A2.BP1</enttypl>
			<enttypd>Raw Cross Power Data File. The header information for this file contains field notes including survey identifier, personnel, date/time, weather, indication of a remote MT

In [42]:
#listing of the SD? files
#Now finally add the processed ASCII text files to the list
sdList = glob.glob(os.path.join(mtStationPath, '*.sd?'),  recursive=True)
#sdList
for i in range(len(sdList)):
  splitSdList = sdList[i].split('\\')
  #fileListing = fileListing + '\t\t\t\t\t\t' + splitSdList[len(splitSdList) - 1] + '\n'
  print ('SD? File:\n' + splitSdList[len(splitSdList) - 1])


SD? File:
CC01A1.SD6
SD? File:
CC01A1.SD7
SD? File:
CC01A1.SD8
SD? File:
CC01A1.SD9
SD? File:
CC01A2.SD6
SD? File:
CC01A2.SD7
SD? File:
CC01A2.SD8
SD? File:
CC01A2.SD9


In [43]:
#SD files Entity and Attributes
for i in range(len(sdList)):
#for i in range(3):
    #if rspList[i].find('BF') > 0: #Do this if the file has a BF style Fixed Width Format
        dfBP = pd.read_fwf(sdList[i], widths=[15,15,15,15,15], skiprows=27, parse_dates=True).rename(columns={'4.3945   1.':'Amp1', '85697    1   2':'Amp2', 'Unnamed: 2':'Amp3', 'Unnamed: 3':'Amp4', 'Unnamed: 4':'Amp5'})
#Now strip out the header rows in the data as to not screw up calculating the range
        #dfSD = dfSD[dfSD.Amp4 != '00000   20   1']
        #print (dfBP)        

sdEandA = ''
sdEandAi = ''
for i in range(len(sdList)):
    splitSdList = sdList[i].split('\\')
    sdEandAi = '\t\t<detailed>\n\t\t<enttyp>\n\t\t\t<enttypl>Text File ' + splitSdList[len(splitSdList) - 1] + '</enttypl>\n' \
        + '\t\t\t<enttypd>Raw Cross Power Data File. The header information for this file contains field notes including survey identifier, personnel, date/time, weather, indication of a remote MT site, additional comments and sensor configuration for the electric and magnetic field channels.</enttypd>' \
        + '\n\t\t\t<enttypds>U.S. Geological Survey</enttypds>\n\t\t</enttyp>' \
        + '\n<attr>\n\t\t\t<attrlabl>Data Value</attrlabl>\n\t\t\t<attrdef>Data Value</attrdef>\n\t\t\t<attrdefs>U.S. Geological Survey</attrdefs>\n\t\t\t<attrdomv>'\
        + '\n\t\t\t\t<udom>Data Value</udom>\n\t\t\t\t</attrdomv>' \
        + '\t\t</attr>\n\t</detailed>\n'
    sdEandA = sdEandA + sdEandAi
    
    
ALL_EandA = ALL_EandA +  sdEandA
#print (len(sdList))
#print ((sdList))
print (sdEandA)

		<detailed>
		<enttyp>
			<enttypl>Text File CC01A1.SD6</enttypl>
			<enttypd>Raw Cross Power Data File. The header information for this file contains field notes including survey identifier, personnel, date/time, weather, indication of a remote MT site, additional comments and sensor configuration for the electric and magnetic field channels.</enttypd>
			<enttypds>U.S. Geological Survey</enttypds>
		</enttyp>
<attr>
			<attrlabl>Data Value</attrlabl>
			<attrdef>Data Value</attrdef>
			<attrdefs>U.S. Geological Survey</attrdefs>
			<attrdomv>
				<udom>Data Value</udom>
				</attrdomv>		</attr>
	</detailed>
		<detailed>
		<enttyp>
			<enttypl>Text File CC01A1.SD7</enttypl>
			<enttypd>Raw Cross Power Data File. The header information for this file contains field notes including survey identifier, personnel, date/time, weather, indication of a remote MT site, additional comments and sensor configuration for the electric and magnetic field channels.</enttypd>
			<enttypds>U.S. Geologi

# Populate Metadata Template

In [44]:
#Load XML Metadata Template File and Read It
metaData = os.path.join(mtMetaDataTemplatePath, mtMetaDataTemplateName)
print ('Metadata path: ' + mtMetaDataTemplatePath + '\n')
xmlTemplateFile = open(metaData, 'r')
metaDataContent = xmlTemplateFile.readlines()
print(metaDataContent)
xmlTemplateFile.close()


Metadata path: C:\CurrentWork\DataReleases\AMT_Silverton

['<?xml version="1.0" encoding="UTF-8"?>\n', '<metadata>\n', '\t<idinfo>\n', '\t\t<citation>\n', '\t\t\t<citeinfo>\n', '<origin>Reitman, J.</origin>\n', '\t\t\t\t<origin>Rodriguez, B. D.</origin>\n', '\t\t\t\t<origin>Brown, P. J.</origin>\t\t\t\t\t\t\t\t\n', '\t\t\t\t<pubdate>2019</pubdate>\n', '\t\t\t\t<title>{title}</title>\n', '\t\t\t\t<edition>1</edition>\n', '\t\t\t\t<geoform>ASCII and Binary Digital Data</geoform>\n', '\t\t\t\t<pubinfo>\n', '\t\t\t\t\t<pubplace>Denver, CO</pubplace>\n', '\t\t\t\t\t<publish>U.S. Geological Survey</publish>\n', '\t\t\t\t</pubinfo>\n', '\t\t\t\t<othercit>Additional information about Originators:Reitman, J., http://orcid.org/0000-0003-3551-9884;Rodriguez, B.D., http://orcid.org/0000-0002-2263-611X; Brown, P.J., http://orcid.org/0000-0002-2415-7462</othercit>\n', '\t\t\t\t<onlink>https://doi.org/10.5066/P9CIAXC5</onlink>\n', '\t\t\t</citeinfo>\n', '\t\t</citation>\n', '\t\t<descript>\n', '\t\t\

In [45]:
# Replace values of current metadata template with the appropriate values.  
# All of this input should have been defined when going through the steps outlined above.
lineString = ''
newMetaDataContent = metaDataContent
splitFileName = ediList[0].split('.')
myfilename = splitFileName[0] + '.xml'
xmlFile = open(myfilename,"w+")
print(myfilename)
#print(keywords.value)
for i in range(len(metaDataContent)):
    lineString = metaDataContent[i]
    if lineString.find('{title}'):
     lineString = lineString.replace('{title}', citTit + '; Station ' + strChildID)
    
    if lineString.find('{abstract}'):
     lineString = lineString.replace('{abstract}', purposeClean)
    
    if lineString.find('{purpose}'):
     lineString = lineString.replace('{purpose}', descriptionClean)
    
    if lineString.find('{BeginFileListingHere}'):
     lineString = lineString.replace('{BeginFileListingHere}', fileListing)
    
    if lineString.find('{keywords}'):
     lineString = lineString.replace('{keywords}', keywords.value)
    
    if lineString.find('{CollectionDate}'):
     lineString = lineString.replace('{CollectionDate}', strCollectionDate)
    
    if lineString.find('{CombinationDate}'):
     lineString = lineString.replace('{CombinationDate}', strCombinationDate)

    if lineString.find('{ConversionDate}'):
     lineString = lineString.replace('{ConversionDate}', strConversionDate)

    if lineString.find('{RotationDate}'):
     lineString = lineString.replace('{RotationDate}', strRotationDate)
    
    if lineString.find('{HarvestDate}'):
     lineString = lineString.replace('{HarvestDate}', strHarvestDate)
   
    if lineString.find('{begdate}'):
     lineString = lineString.replace('{begdate}', begdate)
    
    if lineString.find('{enddate}'):
     lineString = lineString.replace('{enddate}', enddate)
    
    if lineString.find('{SiteLon}'):
     lineString = lineString.replace('{SiteLon}', sitLon)
    
    if lineString.find('{SiteLon}'):
     lineString = lineString.replace('{SiteLat}', sitLat)
    
    if lineString.find('{EandA}'):
     lineString = lineString.replace('{EandA}', ALL_EandA)
    
    if lineString.find('{frequencyMin}'):
     lineString = lineString.replace('{frequencyMin}', str(frequencyMin))
    
    if lineString.find('{frequencyMax}'):
     lineString = lineString.replace('{frequencyMax}', str(frequencyMax))
    
    if lineString.find('{rotationMax}'):
     lineString = lineString.replace('{rotationMax}', str(rotationMax))
    
    if lineString.find('{rotationMin}'):
     lineString = lineString.replace('{rotationMin}', str(rotationMin)) 
    
    if lineString.find('{EdiFileName}'):
     lineString = lineString.replace('{EdiFileName}', EdiFileName)
    
    if lineString.find('{RHOROTmax}'):
     lineString = lineString.replace('{RHOROTmax}', str(RHOROTmax))
    
    if lineString.find('{RHOROTmin}'):
     lineString = lineString.replace('{RHOROTmin}', str(RHOROTmin))
    
    if lineString.find('{RHOXXmax}'):
     lineString = lineString.replace('{RHOXXmax}', str(RHOROTmin))
    
    if lineString.find('{RHOXXmin}'):
     lineString = lineString.replace('{RHOXXmin}', str(RHOXXmin)) 
    
    if lineString.find('{PHSXXmax}'):
     lineString = lineString.replace('{PHSXXmax}', str(PHSXXmax))
    
    if lineString.find('{PHSXXmin}'):
     lineString = lineString.replace('{PHSXXmin}', str(PHSXXmin))
    
    if lineString.find('{RHOXXERRmax}'):
     lineString = lineString.replace('{RHOXXERRmax}', str(RHOXXERRmax))
    
    if lineString.find('{RHOXXERRmin}'):
     lineString = lineString.replace('{RHOXXERRmin}', str(RHOXXERRmin))
    
    if lineString.find('{PHSXXERRmax}'):
     lineString = lineString.replace('{PHSXXERRmax}', str(PHSXXERRmax))
    
    if lineString.find('{PHSXXERRmin}'):
     lineString = lineString.replace('{PHSXXERRmin}', str(PHSXXERRmin))
    
    if lineString.find('{RHOXYmax}'):
     lineString = lineString.replace('{RHOXYmax}', str(RHOXYmax))
    
    if lineString.find('{RHOXYmin}'):
     lineString = lineString.replace('{RHOXYmin}', str(RHOXYmin))
    
    if lineString.find('{PHSXYmax}'):
     lineString = lineString.replace('{PHSXYmax}', str(PHSXYmax))
    
    if lineString.find('{PHSXYmin}'):
     lineString = lineString.replace('{PHSXYmin}', str(PHSXYmin))
    
    if lineString.find('{RHOXYERRmax}'):
     lineString = lineString.replace('{RHOXYERRmax}', str(RHOXYERRmax))
    
    if lineString.find('{RHOXYERRmin}'):
     lineString = lineString.replace('{RHOXYERRmin}', str(RHOXYERRmin))
    
    if lineString.find('{PHSXYERRmax}'):
     lineString = lineString.replace('{PHSXYERRmax}', str(PHSXYERRmax))
    
    if lineString.find('{PHSXYERRmin}'):
     lineString = lineString.replace('{PHSXYERRmin}', str(PHSXYERRmin))
    
    if lineString.find('{RHOYXmax}'):
     lineString = lineString.replace('{RHOYXmax}', str(RHOYXmax))
    
    if lineString.find('{RHOYXmin}'):
     lineString = lineString.replace('{RHOYXmin}', str(RHOYXmin))
    
    if lineString.find('{PHSYXmax}'):
     lineString = lineString.replace('{PHSYXmax}', str(PHSYXmax))
    
    if lineString.find('{PHSYXmin}'):
     lineString = lineString.replace('{PHSYXmin}', str(PHSYXmin))
    
    if lineString.find('{RHOYXERRmax}'):
     lineString = lineString.replace('{RHOYXERRmax}', str(RHOYXERRmax))
    
    if lineString.find('{RHOYXERRmin}'):
     lineString = lineString.replace('{RHOYXERRmin}', str(RHOYXERRmin))
    
    if lineString.find('{PHSYXERRmax}'):
     lineString = lineString.replace('{PHSYXERRmax}', str(PHSYXERRmax))
    
    if lineString.find('{PHSYXERRmin}'):
     lineString = lineString.replace('{PHSYXERRmin}', str(PHSYXERRmin))
    
    if lineString.find('{RHOYYmax}'):
     lineString = lineString.replace('{RHOYYmax}', str(RHOYYmax))
    
    if lineString.find('{RHOYYmin}'):
     lineString = lineString.replace('{RHOYYmin}', str(RHOYYmin))
    
    if lineString.find('{PHSYYmax}'):
     lineString = lineString.replace('{PHSYYmax}', str(PHSYYmax))
    
    if lineString.find('{PHSYYmin}'):
     lineString = lineString.replace('{PHSYYmin}', str(PHSYYmin))
    
    if lineString.find('{RHOYYERRmax}'):
     lineString = lineString.replace('{RHOYYERRmax}', str(RHOYYERRmax))
    
    if lineString.find('{RHOYYERRmin}'):
     lineString = lineString.replace('{RHOYYERRmin}', str(RHOYYERRmin))
    
    if lineString.find('{PHSYYERRmax}'):
     lineString = lineString.replace('{PHSYYERRmax}', str(PHSYYERRmax))
    
    if lineString.find('{PHSYYERRmin}'):
     lineString = lineString.replace('{PHSYYERRmin}', str(PHSYYERRmin))
    
    if lineString.find('{TIPMAGmax}'):
     lineString = lineString.replace('{TIPMAGmax}', str(TIPMAGmax))
    
    if lineString.find('{TIPMAGmin}'):
     lineString = lineString.replace('{TIPMAGmin}', str(TIPMAGmin))
    
    if lineString.find('{TIPPHSmax}'):
     lineString = lineString.replace('{TIPPHSmax}', str(TIPPHSmax))
    
    if lineString.find('{TIPPHSmin}'):
     lineString = lineString.replace('{TIPPHSmin}', str(TIPPHSmin))
    
    if lineString.find('{TipperAttributes}'):
     lineString = lineString.replace('{TipperAttributes}', str(TipperAttributes))
    
    if lineString.find('{ImpedanceAttributes}'):
     lineString = lineString.replace('{ImpedanceAttributes}', str(ImpedanceAttributes))
    
    if lineString.find('{TIPMAGERRmax}'):
     lineString = lineString.replace('{TIPMAGERRmax}', str(TIPMAGERRmax))
    
    if lineString.find('{TIPMAGERRmin}'):
     lineString = lineString.replace('{TIPMAGERRmin}', str(TIPMAGERRmin))
    
    if lineString.find('{TIPPHSERRmax}'):
     lineString = lineString.replace('{TIPPHSERRmax}', str(TIPPHSERRmax))
    
    if lineString.find('{TIPPHSERRmin}'):
     lineString = lineString.replace('{TIPPHSERRmin}', str(TIPPHSERRmin))
    
    if lineString.find('{ZSTRIKEmax}'):
     lineString = lineString.replace('{ZSTRIKEmax}', str(ZSTRIKEmax))
    
    if lineString.find('{ZSTRIKEmin}'):
     lineString = lineString.replace('{ZSTRIKEmin}', str(ZSTRIKEmin))
    
    if lineString.find('{ZSKEWmax}'):
     lineString = lineString.replace('{ZSKEWmax}', str(ZSKEWmax))
    
    if lineString.find('{ZSKEWmin}'):
     lineString = lineString.replace('{ZSKEWmin}', str(ZSKEWmin))
    
    if lineString.find('{TSTRIKEmax}'):
     lineString = lineString.replace('{TSTRIKEmax}', str(TSTRIKEmax))
    
    if lineString.find('{TSTRIKEmin}'):
     lineString = lineString.replace('{TSTRIKEmin}', str(TSTRIKEmin))
    
    if lineString.find('{COHmax}'):
     lineString = lineString.replace('{COHmax}', str(COHmax))
    
    if lineString.find('{COHmin}'):
     lineString = lineString.replace('{COHmin}', str(COHmin))
    
    if lineString.find('{EPREDCOHmax}'):
     lineString = lineString.replace('{EPREDCOHmax}', str(EPREDCOHmax))
    
    if lineString.find('{EPREDCOHmin}'):
     lineString = lineString.replace('{EPREDCOHmin}', str(EPREDCOHmin))
    
    if lineString.find('{SIGAMPmax}'):
     lineString = lineString.replace('{SIGAMPmax}', str(SIGAMPmax))
    
    if lineString.find('{SIGAMPmin}'):
     lineString = lineString.replace('{SIGAMPmin}', str(SIGAMPmin))
    
    if lineString.find('{SIGNOISEmax}'):
     lineString = lineString.replace('{SIGNOISEmax}', str(SIGNOISEmax))
    
    if lineString.find('{SIGNOISEmin}'):
     lineString = lineString.replace('{SIGNOISEmin}', str(SIGNOISEmin))
   
    # {county}

    else:
     lineString = lineString
    xmlFile.write(lineString)
    
    #print (lineString)
     
    
    
#for r in (metaDataContent):
    #newMetaDataContent = metaDataContent.replace('{title}', drTitle)
    #newMetaDataContent = metaDataContent.replace('{keywords}', keywords.value)
xmlFile.close()

print ('Creation of new metadata file is complete\n\n') 
#Load EDI File and Read It
##checkFile = open(open(myfilename, 'r')
##checkFileContent = checkFile.read()
##checkFile.close()
##print(checkFileContent)

C:\CurrentWork\DataReleases\AMT_Silverton\CC01\USA-Colorado-Silverton_Caldera-2018-AMTCC01.xml
Creation of new metadata file is complete




### Check this file to see if it is valid against the FGDC metadata standard (FGDC-STD-001-1998)

## https://mrdata.usgs.gov/validation/

In [46]:
# Show the resulting child xml metadata file example 
#for i in range(len(newMetaDataContent)):
print (newMetaDataContent)

['<?xml version="1.0" encoding="UTF-8"?>\n', '<metadata>\n', '\t<idinfo>\n', '\t\t<citation>\n', '\t\t\t<citeinfo>\n', '<origin>Reitman, J.</origin>\n', '\t\t\t\t<origin>Rodriguez, B. D.</origin>\n', '\t\t\t\t<origin>Brown, P. J.</origin>\t\t\t\t\t\t\t\t\n', '\t\t\t\t<pubdate>2019</pubdate>\n', '\t\t\t\t<title>{title}</title>\n', '\t\t\t\t<edition>1</edition>\n', '\t\t\t\t<geoform>ASCII and Binary Digital Data</geoform>\n', '\t\t\t\t<pubinfo>\n', '\t\t\t\t\t<pubplace>Denver, CO</pubplace>\n', '\t\t\t\t\t<publish>U.S. Geological Survey</publish>\n', '\t\t\t\t</pubinfo>\n', '\t\t\t\t<othercit>Additional information about Originators:Reitman, J., http://orcid.org/0000-0003-3551-9884;Rodriguez, B.D., http://orcid.org/0000-0002-2263-611X; Brown, P.J., http://orcid.org/0000-0002-2415-7462</othercit>\n', '\t\t\t\t<onlink>https://doi.org/10.5066/P9CIAXC5</onlink>\n', '\t\t\t</citeinfo>\n', '\t\t</citation>\n', '\t\t<descript>\n', '\t\t\t<abstract>This dataset includes audiomagnetotelluric (AMT